<a href="https://colab.research.google.com/github/AlexWalcher/optimizationOfHintGeneration/blob/Test/optimizationOfHintGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##How to use:
1. Run **All imports**
2. Run **Load File**
3. Run **New prediction method for years** or any of the **Old predictions methods**

# **All imports**

In [101]:
%cd /content/
%rm -r optimizationOfHintGeneration
!git clone https://github.com/AlexWalcher/optimizationOfHintGeneration.git
%cd /content/

/content
Cloning into 'optimizationOfHintGeneration'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 123 (delta 12), reused 0 (delta 0), pack-reused 85
Receiving objects: 100% (123/123), 7.21 MiB | 19.79 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content


In [102]:
!pip install selenium
!apt-get update 
!apt-get install firefox
!apt install firefox-geckodriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,343 kB]
Get:

In [103]:
!pip install sparqlwrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
!pip install pageviewapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [105]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
import pandas as pd
pd.set_option('display.max_colwidth',1000)

In [108]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")


In [109]:
import requests

In [110]:
import bs4
from bs4 import BeautifulSoup, NavigableString, Tag

In [111]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [112]:
import wikipedia

In [113]:
import random

In [114]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy link en_core_web_sm en
import spacy

In [115]:
#new imports

import time
import requests
import re
import difflib
import pprint


from selenium import webdriver
from bs4 import BeautifulSoup

from urllib.parse import urlparse, parse_qs
from collections import OrderedDict

In [116]:
#import collections.abc as collections
#from collections import Mapping

!pip install Wikipedia-API
import wikipediaapi

!pip install git+https://github.com/Commonists/pageview-api.git
#import pageviewapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Commonists/pageview-api.git to /tmp/pip-req-build-hzz85v_g
  Running command git clone --filter=blob:none --quiet https://github.com/Commonists/pageview-api.git /tmp/pip-req-build-hzz85v_g
  Resolved https://github.com/Commonists/pageview-api.git to commit 39e8b3c3c82f64a500e3dd4f306451c81c7e31b7
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for

# **Load File**

In [117]:
import pandas as pd

df = pd.ExcelFile("./optimizationOfHintGeneration/testSet.xlsx").parse("Sheet1")
x = []
x.append(df["Answer"])

dataPerson = []
dataYear = []
dataLocation = []

for index, row in df.iterrows():
  if(row["Category"] == "Person"):
    dataPerson.append([row["Question"], row["Answer"]])
  elif(row["Category"] == "Year"):
    dataYear.append([row["Question"], row["Answer"]])
  elif(row["Category"] == "Location"):
    dataLocation.append([row["Question"], row["Answer"]])

person_df = pd.DataFrame(dataPerson, columns=["Question", "Answer"])
year_df = pd.DataFrame(dataYear, columns=["Question", "Answer"])
location_df = pd.DataFrame(dataLocation, columns=["Question", "Answer"])

# **New prediction method for years**

## Dictionary for thumbcaption part of a year



In [118]:
def get_table_info(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    options = webdriver.FirefoxOptions()
    #options.headless = True
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(5) # Wait for the page to load completely
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])
    return (headers, data)

def get_table_info_requests(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    headers = []
    data = []

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])
    return (headers, data)


def get_links_in_section(wikipedia_url, section_heading):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link on that section of the page.
    """
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('span', {'id': section_heading})
    if section is None:
        return None
    section_content = section.parent.find_next_sibling('ul')
    if section_content is None:
        return None
    links = []
    for item in section_content.find_all('li'):
        link = item.find('a')
        if link is not None and link.has_attr('href') and link['href'].startswith('/wiki/'):
            title = link.get('title', '')
            description = item.text.strip()
            url = f"https://en.wikipedia.org{link['href']}"
            links.append({'title': title, 'description': description, 'url': url})
    print(links)
    return links


def get_links_in_section_with_sublinks(wikipedia_url, section_title):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link  with its sublinks as well.
    """
    response = requests.get(wikipedia_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        section_heading = soup.find('span', {'id': section_title})
        if section_heading is None:
            return None
        section = section_heading.parent
        section_content = section.find_next_sibling('ul')
        if section_content is None:
            return None
        links = []
        for item in section_content.find_all('li'):
            link = item.find('a')
            if link is not None:
                link_title = link.get('title')
                link_url = 'https://en.wikipedia.org' + link.get('href')
                link_description = item.text.replace(link_title, '').strip()
                links.append({'title': link_title, 'url': link_url, 'description': link_description})
                for sublink in item.find_all('a', href=True, recursive=False):
                    sublink_title = sublink.get('title')
                    sublink_url = 'https://en.wikipedia.org' + sublink.get('href')
                    sublink_description = sublink.parent.text.replace(sublink_title, '').replace(link_title, '').strip()
                    links.append({'title': sublink_title, 'url': sublink_url, 'description': sublink_description})
        return links
    else:
        return None

"""
Returns all backlinks located in the thumbcaption section
"""
def get_wikipedia_backlinks_thumbcaption(url):
    # Load the Wikipedia page HTML
    page_html = requests.get(url).text
    soup = BeautifulSoup(page_html, 'html.parser')

    # Find the image caption on the page
    caption = soup.find('div', class_='thumbcaption')

    if caption is not None:
      # Extract the caption text and any backlinks
      backlink_sentences = {}
      sentences = caption.text.strip().split('.')
      for sentence in sentences:
        links = caption.find_all('a', href=True, string=re.compile(sentence))
        if len(links) > 0:
            backlinks = []
            for link in links:
                backlink_url = 'https://en.wikipedia.org' + link['href']
                backlink_title = link.get('title', '')
                backlinks.append((backlink_url, backlink_title))
            backlink_sentences[sentence] = backlinks
    return backlink_sentences

"""
This function correctly prunes the links to only include the string after the last / character
"""
def prune_links(links):
    pruned_links = []
    for url, title in links:
        pruned_url = url.split('/')[-1]
        pruned_links.append((pruned_url, title))
    return pruned_links

"""
This function first initializes an empty list combined_list that will hold the combined strings. Then, it uses a for loop to iterate through the input list in increments of 10 tuples at a time.
For each sub-list of up to 10 tuples, it uses the list comprehension and join() method as before to combine the first elements of each tuple into a single string separated by '|'. 
Finally, the function appends each combined string to the combined_list and returns it at the end.
"""
def combine_first_elements(my_list):
    combined_list = []
    num_tuples = len(my_list)
    for i in range(0, num_tuples, 10):
        sub_list = my_list[i:i+10]
        combined_str = '|'.join([tup[0] for tup in sub_list])
        combined_list.append(combined_str)
    return combined_list

"""
This function first initializes an empty list url_list that will hold the modified URLs. Then, it uses a for loop to iterate through each combined string in the input list. 
For each combined string, it concatenates the base URL with a forward slash and the combined string, and appends the resulting URL to the url_list. Finally, the function returns the url_list at the end.
"""
def add_combined_strings_to_url(base_url, combined_strings):
    url_list = []
    for string in combined_strings:
        url_list.append(base_url + '/' + string)
    return url_list


"""
This will output a list of all the sentences in the thumbcaption, split by ';'.
"""
def get_thumbcaption_sentences(url):
    # Get the HTML content of the page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find the thumbcaption element
    thumbcaption = soup.find('div', class_='thumbcaption')

    # Get all the sentences in the thumbcaption
    sentences = thumbcaption.text.split('; ')

    return sentences

"""
This function converts a list into a dict in the way that is needed.
"""
def list_to_dict(lst):
    result = {}
    for sublist in lst:
        if len(sublist) >= 4:
            key = sublist[1]
            value = int(sublist[3].replace(',', ''))
            result[key] = value
    return result

"""
This function takes a list of links, opens each link to get the data, discards the header and converts the data into a dictionary using the list_to_dict() function, 
and then updates a combined dictionary with the resulting dictionary from each link. Finally, it returns the combined dictionary.
"""
def combine_dicts_from_links(link_list):
    combined_dict = {}
    for link in link_list:
        header, data = get_table_info(link)
        #print(data)
        link_dict = list_to_dict(data)
        combined_dict.update(link_dict)
    return combined_dict

"""
This sorts the items in the dictionary based on the integer value of the second element in each key-value tuple (i.e. item[1]), in descending order (reverse=True).
"""
def sort_dict_desc(d):
    return {k: v for k, v in sorted(d.items(), key=lambda item: int(item[1]), reverse=True)}

"""
This function takes in the ord dictionary and the sentences list as arguments.
It initializes an empty list called result that we will append the found sentences to. It then iterates over each key in the ord dictionary and for each key, it iterates over each sentence in the sentences list. 
If the key is found in the sentence, the sentence is appended to the result list
"""
def find_sentences(ord, sentences):
    result = []
    for key in ord:
        for sentence in sentences:
            if key in sentence:
                result.append(sentence)
    return result

"""
This function takes a list of sentences and a keyword, removes the keyword from each sentence in the list, and returns the updated list.
"""
def remove_keyword(sentences, keyword):
    updated_sentences = []
    for sentence in sentences:
        updated_sentence = sentence.replace(keyword, "")
        updated_sentences.append(updated_sentence)
    return updated_sentences

"""
This function takes a list of sentences and a list of keyword, removes the keyword from each sentence in the list, and returns the updated list. MAYBE NOT WORKING
"""
def remove_keywords(sentences, keywords):
    """
    Removes one or more keywords from each sentence in the list of sentences.
    """
    result = sentences
    for entry in keywords:
        result = remove_keyword(result, entry)
    
    return result
"""
This function prepends a given string to each sentence in a list.
"""
def prepend_string(sentences, prepend_str):
    new_sentences = []
    for sentence in sentences:
        new_sentence = f"{prepend_str}{sentence}"
        new_sentences.append(new_sentence)
    return new_sentences

"""
This function creates a new list new_sentences and loops over each sentence in the input sentences list. 
It then uses a list comprehension and the difflib.SequenceMatcher class to compare the sentence to each sentence already in new_sentences. 
If the ratio of similarity between the two sentences is greater than 0.8 (adjust this threshold as needed), it considers the sentence to be similar and skips it. Otherwise, it adds the sentence to new_sentences. 
Finally, it returns the new list of unique sentences.
"""


def remove_similar(sentences):
    new_sentences = []
    for sentence in sentences:
        if not any(difflib.SequenceMatcher(None, sentence, s).ratio() > 0.8 for s in new_sentences):
            new_sentences.append(sentence)
    return new_sentences


In [119]:
#IMPORTANT
def thumbcaption_hints_per_year(years_list):
  thumbcaption_hints = {}
  wiki_base_link= 'https://en.wikipedia.org/wiki/'
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=all-time&pages='
  
  for y in years_list:
    years_key = str(y)
    test_link = wiki_base_link + years_key
    sentences_of_thumbcaption = get_thumbcaption_sentences(test_link) #just gets the sentences from the thumbcaption section of a wikipedi years page 
    thumbcaption = get_wikipedia_backlinks_thumbcaption(test_link) #get all backlinks of the thumbcapture of the year (those are the most known events)
    thumbcaption_key = next(iter(thumbcaption))
    thumbcaption_val = thumbcaption[thumbcaption_key]
    pruned = prune_links(thumbcaption_val) #prune those backlinks such that only the important part remains
    com = combine_first_elements(pruned) #combine up to 10 of these links to create a request to pageview
    url_list = add_combined_strings_to_url(pageviews_range_url, com) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page
    print("URL")
    print(url_list)
    data=combine_dicts_from_links(url_list) #now we called the links and retreieved the pageviews; saved them as a dictionary
    ord = sort_dict_desc(data) #now the list is ordered in ascending order
    tmp = find_sentences(ord,sentences_of_thumbcaption) #search the corresponding sentence to the keyword (USA and school shooting for example)
    #remove the years number from the hints OBVIOUSNESS
    keywords_list = [years_key, 'clockwise ', 'Clockwise ', 'From top left', 'from top-left', 'from top left', 'From top-left', 'from top-left: ', ':', 'from left, clockwise'] #list of keywords that should be removed from the sentences
    t4=remove_keywords(tmp, keywords_list)
    prepend_str = 'In the same year, '
    hints = prepend_string(t4, prepend_str)

    final_hints = remove_similar(hints) #before adjusting the sentences
    thumbcaption_hints[y] = final_hints

  return thumbcaption_hints

def get_year_thumbcaption_hints():
  file_years_list = []
  for index, row in year_df.iterrows():
    file_years_list.append(row["Answer"])

  pop_thumb_hints = thumbcaption_hints_per_year(file_years_list)
  return pop_thumb_hints

#get_year_thumbcaption_hints()

## Dictionary for popular sports events of a year

In [120]:
#Some global varaibles for faster execution
pop_year_hints = {}
pop_thumb_hints = {}

def get_all_tables(url):
    """
    Given a URL, this function opens the url and retrieves all tables on the page.
    """
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(1) # Wait for the page to load completely
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    tables = soup.find_all('table')
    all_tables = []
    for table in tables:
        rows = table.find_all('tr')
        headers = [header.text.strip() for header in rows[0].find_all('th')]
        data = []
        for row in rows[1:]:
            data.append([cell.text.strip() for cell in row.find_all('td')])
        all_tables.append({'headers': headers, 'data': data})

    return all_tables

def get_first_elements(data_dict):
    """
    Extracts the first element from each sublist in the 'data' list of lists in a dictionary and returns them in a separate list.
    
    Args:
    - data_dict (dict): A dictionary containing a 'data' key with a list of lists as its value.
    
    Returns:
    - A list containing the first element of each sublist in the 'data' list of lists.
    """
    data_list = data_dict['data']
    result = []
    for sublist in data_list:
        if len(sublist) > 0:
            if len(sublist[0]) > 0:
              result.append(sublist[0])
    return result

#prune the list of the hole wiki page down to the important table
def prune_dict_list(dict_list, keyw):
    for d in dict_list:
        if 'headers' in d and d['headers'] == [keyw]:
            return d
    return None  # if no dict with the desired key-value pair is found

#replace the \n and create a list of lists 
def create_list_from_list_of_lists_key(lst, keyw):
    result = []
    for sublst in lst:
        if sublst:
          new_test = [item.replace(keyw, '') for item in sublst[0].split(keyw)]
          result.append(new_test)
    return result

#replace the : and create a dict; {'year': 'CL-winner'}
def create_dict_from_list_of_lists(lst):
    result_dict = {}
    for sublist in lst:
        for item in sublist:
            if ":" in item:
                key, value = item.split(":")
                key = key.strip().replace("–", "-")
                value = value.strip()
                result_dict[key] = value
            else:
                continue
    return result_dict

#split on the : and creates a dict
def create_dict_from_list(lst):
    result = {}
    for item in lst:
        parts = item.split(":")
       # parts = parts.strip().replace(":", "")
        result[parts[0]] = parts[1]
    return result

#splits on [ and creates a dict 1950: 'Farina'
def get_year_with_driver(race_results):
    results_dict = {}
    for result in race_results:
        if result:
            results_dict[result[0].split("[")[0]] = result[1]
    return results_dict

#get rid of the links ('Alberto Ascari[20]' => 'Alberto Ascari')
def clean_driver_names(results_dict):
    for year, driver in results_dict.items():
        results_dict[year] = driver.split('[')[0].strip()
    return results_dict

#deletes the : from the key
def clean_dict_keys(dict_to_clean):
    cleaned_dict = {}
    for key, value in dict_to_clean.items():
        cleaned_dict[key.rstrip(':')] = value
    return cleaned_dict

#function to split on \n but that lets the city names stay together
def create_city_dict(city_list):
    city_dict = {}
    cities = city_list[0].split('\n')
    for city in cities:
        if city:
            year, *city_name = city.strip().split()
            city_dict[year] = ' '.join(city_name)
    return city_dict


#get the dict of all the champions league winners 
def champions_league_winners_list():
    champions_league_url = 'https://en.wikipedia.org/wiki/List_of_European_Cup_and_UEFA_Champions_League_finals#List_of_finals'
    all = get_all_tables(champions_league_url) #gets all tables of wiki page
    #first prune of that huge dict
    keyw= 'showvteEuropean Cup and UEFA Champions League winners'
    pruned_dict = prune_dict_list(all,keyw)
    #second prune
    inter = pruned_dict.get('data') 
    cl_list = inter[2:5] + inter[7:11]
    tmp1 = create_list_from_list_of_lists_key(cl_list, '\n')
    tmp2 = create_dict_from_list_of_lists(tmp1)

    return tmp2

#get the dict of all the euros winners 
def uefa_euros_winners_list():
    euros_url = 'https://en.wikipedia.org/wiki/List_of_UEFA_European_Championship_finals#List_of_finals'
    all = get_all_tables(euros_url) #gets all tables of wiki page
    #first prune of that huge dict
    keyw= 'showvteUEFA European Championship winners'
    pruned_dict = prune_dict_list(all,keyw)
    #second prune
    inter = pruned_dict.get('data') 
    tmp1 = create_list_from_list_of_lists_key(inter, '\n')
    tmp2 = create_dict_from_list_of_lists(tmp1)

    return tmp2

#get the dict of all the wold cup winners 
def uefa_worlds_winners_list():
    worlds_url = 'https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals#List_of_final_matches'
    all = get_all_tables(worlds_url) #gets all tables of wiki page
    tmp3=get_first_elements(all[3])
    #first prune of that huge dict
    keyw= 'showvteFIFA World Cup'
    pruned_dict = prune_dict_list(all,keyw)
    #second prune
    inter = pruned_dict.get('data') 
    inter = inter[2]
    years = [s for s in inter[0].split('\n')]
    my_dict = dict(zip(years, tmp3))

    return my_dict

#get the dict of all the F1 drivers world champions
def f1_winners_list():
    euros_url = 'https://en.wikipedia.org/wiki/List_of_Formula_One_World_Drivers%27_Champions#By_season'
    all = get_all_tables(euros_url) #gets all tables of wiki page
    lst = all[2].get('data')
    tmp2 = get_year_with_driver(lst)
    tmp2 = clean_driver_names(tmp2)

    return tmp2

#get the dict of all summer olympics host cities
def summer_olympics_hosts_list():
    summerO_url = 'https://en.wikipedia.org/wiki/Summer_Olympic_Games#List_of_Summer_Olympic_Games'
    all = get_all_tables(summerO_url) #gets all tables of wiki page
    lst = all[10].get('data')
    tmp1 = create_list_from_list_of_lists_key(lst, '\n')
    tmp1 = tmp1[0]
    tmp1 = create_dict_from_list(tmp1)
    tmp1 = clean_dict_keys(tmp1)
    tmp1 = clean_driver_names(tmp1)

    return tmp1

#get the dict of all winter olympics host cities
def winter_olympics_hosts_list():
    winterO_url = 'https://en.wikipedia.org/wiki/Winter_Olympic_Games#List_of_Winter_Olympic_Games'
    all = get_all_tables(winterO_url) #gets all tables of wiki page
    lst = all[8].get('data')
    tmp1 = create_list_from_list_of_lists_key(lst, '\n')
    tmp1 = tmp1[0]
    tmp1 = create_dict_from_list(tmp1)
    tmp1 = clean_dict_keys(tmp1)
    tmp1 = clean_driver_names(tmp1)

    return tmp1

In [121]:
#The basic sentences out of which we create the hint-sentences
basic_sentences =  ['In the same year, ', 'In the previous year, ', 'In the following year, ']
sport_sentences = [' has won the UEFA Champions League.', ' has won the UEFA Euro Football Championship.', ' has won the FIFA World Cup.', ' has won the F1 Drivers World Championship.', ]
olympic_sentences = ['In the same year, the Summer Olympics were held in ', 'In the previous year, the Summer Olympics were held in ', 'In the following year, the Summer Olympics were held in ', 'In the same year, the Winter Olympics were held in ', 'In the previous year, the Winter Olympics were held in ', 'In the following year, the Winter Olympics were held in ']

#write all of the winners of the different sports categories into lists
cl_all = champions_league_winners_list() #For the Champions league, 
euro_all = uefa_euros_winners_list() #For Football-Euros
worlds_all = uefa_worlds_winners_list() #For Football-Worlds
f1_all = f1_winners_list() #For Fromula1
summer_olympics_all = summer_olympics_hosts_list() #For OlympicSummerGames 
winter_olympics_all = winter_olympics_hosts_list() #For OlympicWinterGames 

'''
takes a list of years and then creates a dict of dicts, where (if available) the most popular sports events of that year are saved as hints.
returns a dict with the corresponding sports events from the years in years_list
'''
def popular_sports_per_year(years_list):
 
  pop_sport_hints_year = {}
  for index in years_list:
    year = index
    year_s = str(year)

    year_dict = {
        'cl': '', 'p_cl': '', 'f_cl': '',
        'euros': '', 'p_euros': '', 'f_euros': '',
        'worlds': '', 'p_worlds': '', 'f_worlds': '',
        'f1': '', 'p_f1': '', 'f_f1': '',
        'summer': '', 'p_summer': '', 'f_summer': '',
        'winter': '', 'p_winter': '', 'f_winter': '', 
        }

  # UEFA Champions League: Create the sentences like (In the same-, the following-, the previous-year)
    for key in cl_all:
      if int(key.split('-')[1]) == year % 100:
        result = cl_all[key]
        break
    if result:
      year_dict['cl'] = basic_sentences[0] + result + sport_sentences[0] 
    for key in cl_all:
      if int(key.split('-')[1]) == (year - 1) % 100:
        result = cl_all[key]
        break
    if result:
      year_dict['p_cl'] = basic_sentences[1] + result + sport_sentences[0] 
    for key in cl_all:
      if int(key.split('-')[1]) == (year + 1) % 100:
        result = cl_all[key]
        break
    if result:
      year_dict['f_cl'] = basic_sentences[2] + result + sport_sentences[0]       

  # UEFA EURO Football Championship: Create the sentences like (In the same-, the following-, the previous-year)
    for d in euro_all:
      if year_s in d:
        year_dict['euros'] = basic_sentences[0] + euro_all[year_s] + sport_sentences[1]       
    for d in euro_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_euros'] = basic_sentences[1] + euro_all[year_int] + sport_sentences[1]
    for d in euro_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_euros'] = basic_sentences[2] + euro_all[year_int] + sport_sentences[1]

  # FIFA WORLD Football Championship: Create the sentences like (In the same-, the following-, the previous-year)
    for d in worlds_all:
      if year_s in d:
        year_dict['worlds'] = basic_sentences[0] + worlds_all[year_s] + sport_sentences[2]
    for d in worlds_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_worlds'] = basic_sentences[1] + worlds_all[year_int] + sport_sentences[2]
    for d in worlds_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_worlds'] = basic_sentences[2] + worlds_all[year_int] + sport_sentences[2]

  # F1 WORLD Drivers Championship: Create the sentences like (In the same-, the following-, the previous-year)
    for d in f1_all:
      if year_s in d:
        year_dict['f1'] = basic_sentences[0] + f1_all[year_s] + sport_sentences[3]
    for d in f1_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_f1'] = basic_sentences[1] + f1_all[year_int] + sport_sentences[3]  
    for d in f1_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_f1'] = basic_sentences[2] + f1_all[year_int] + sport_sentences[3]    

  # Summer Olympic Games: Create the sentences like (In the same-, the following-, the previous-year)
    for d in summer_olympics_all:
      if year_s in d:
        year_dict['summer'] = olympic_sentences[0] + summer_olympics_all[year_s] 
    for d in summer_olympics_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_summer'] = olympic_sentences[1] + summer_olympics_all[year_int] 
    for d in summer_olympics_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_summer'] = olympic_sentences[2] + summer_olympics_all[year_int]  

  # Winter Olympic Games: Create the sentences like (In the same-, the following-, the previous-year)
    for d in winter_olympics_all:
      if year_s in d:
        year_dict['winter'] = olympic_sentences[3] + winter_olympics_all[year_s] 
    for d in winter_olympics_all:
      t = year - 1
      year_int = str(t)
      if year_int in d:
        year_dict['p_winter'] = olympic_sentences[4] + winter_olympics_all[year_int] 
    for d in winter_olympics_all:
      t = year + 1
      year_int = str(t)
      if year_int in d:
        year_dict['f_winter'] = olympic_sentences[5] + winter_olympics_all[year_int]  

    #write the entry in the dict
    pop_sport_hints_year[year] = year_dict
  
  return pop_sport_hints_year

#CALL FUNCTION
#returns a dictionary of the years that occured in the xls file, with its corresponding hints from the most popular sports events of that year.
def get_year_sports_hints():
  file_years_list = []
  for index, row in year_df.iterrows():
    file_years_list.append(row["Answer"])
  pop_sport_hints = popular_sports_per_year(file_years_list)
  
  return pop_sport_hints


In [122]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def preprocess_text(text):
    # Tokenize the text and add special tokens
    tokens = tokenizer.encode(text, add_special_tokens=True)
    # Convert the tokens to a tensor
    token_tensor = torch.tensor(tokens).unsqueeze(0)
    return token_tensor

def get_similarity_score(text1, text2):
    # Preprocess both texts
    tensor1 = preprocess_text(text1)
    tensor2 = preprocess_text(text2)

    # Pass both tensors to the model to get the embeddings
    with torch.no_grad():
        output1 = model(tensor1)
        output2 = model(tensor2)
    
    # Compute the cosine similarity between the two embeddings
    cosine_sim = torch.nn.functional.cosine_similarity(output1.last_hidden_state.mean(dim=1), output2.last_hidden_state.mean(dim=1), dim=1)
    return cosine_sim.item()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Test it!

In [123]:
# create the hints for the questions where the answer is a year and print them
#if len(pop_year_hints) == 0 and len(pop_thumb_hints) == 0:
pop_year_hints = get_year_sports_hints()
pop_thumb_hints = get_year_thumbcaption_hints()

years_hints = {}

for y in pop_year_hints:
  year_dict = {
      'sports': pop_year_hints[y],
      'thumbcaption': pop_thumb_hints[y]
  }

  years_hints[y] = year_dict

generated_hints_for_years = years_hints
pprint.pprint(generated_hints_for_years, indent=1)

'''
#Test for utility score of new questions; calculate score via BERT for each question,hint pair and write the score together with the question into the sim_scores dictionary.
'''

qa_dict = dict(zip(year_df['Answer'], year_df['Question']))
sim_scores = years_hints

for y, q in qa_dict.items():
  for year, data in years_hints.items():
    if y == year:
      for category, subdata in data.items():
        #sim_scores[year]['question'] = q
        if category != 'thumbcaption':
          for key, value in subdata.items():
            sim_scores[year][category][key] = {}
            similarity_score = get_similarity_score(q,value)
            sim_scores[year][category][key][value] = similarity_score
        else:
          for i in subdata:
            sim_scores[year][category] = {}
            similarity_score = get_similarity_score(q,i)
            sim_scores[year][category][i] = similarity_score
    else:
      continue

for y, q in qa_dict.items():
  for year, data in years_hints.items():
    if y == year:
      sim_scores[year]['question'] = q

pprint.pprint(sim_scores, indent=1)

{}
{}


## Dictionary for thumbcaption part of a year



# **New prediction Methods for locations and people:**

## All the functions to get wiki-categories of a location-, or person-question

We want to create some sort of metric to explain how popular or well-known a entity is, to give better hints. Therefore we look at how many options there are (how many entries in a certain category) and at how diverse these options are. If a category for example has 100s of entries, then it probably isn't a good base for a hint because there are many to choose from.



In [124]:
def get_category_links(categories):
    """
    Given a list of Wikipedia category names, creates the corresponding Wikipedia category links.
    
    Args:
        categories (list): A list of Wikipedia category names to create links for.
    
    Returns:
        category_links (list): A list of Wikipedia category links corresponding to the input categories.
    """
    category_links = []
    for category in categories:
        category_link = "https://en.wikipedia.org/wiki/Category:" + category.replace(" ", "_")
        #category_link = "https://wikipedia.org/wiki/Category:" + category.replace(" ", "_")
        category_links.append(category_link)
    return category_links

def get_category_with_underscores(categories):
    """
    Given a list of Wikipedia category names, creates the corresponding Wikipedia category links.
    Args: categories (list): A list of Wikipedia category names to create links for.
    Returns: category_links (list): A list of Wikipedia category links corresponding to the input categories.
    """
    category_links = []
    for category in categories:
        category_link = category.replace(" ", "_")
        category_links.append(category_link)
    return category_links

def get_category_entry_counts(categories):
    """
    Given a list of Wikipedia category names, retrieves the number of entries in each category and returns a dictionary
    with the category names as keys and the entry counts as values.
    Args: categories (list): A list of Wikipedia category names to retrieve entry counts for.
    Returns: category_entry_counts (dict): A dictionary of Wikipedia category names and their respective entry counts.
    """
    category_links = get_category_links(categories)
    category_entry_counts = {}

    for i, category_link in enumerate(category_links):
        response = requests.get(category_link)
        soup = BeautifulSoup(response.content, "html.parser")

        try:
            entry_count_text = soup.find("div", {"id": "catlinks"}).find_all("a")[1].text.strip()
            entry_count = int(entry_count_text.split()[-2])
            category_name = categories[i]
            if category_name:
                category_entry_counts[category_name] = entry_count
        except:
            pass

    return category_entry_counts

In [125]:
#rewrite with wikiapi


def get_wikipedia_categories(title):
    """
    Retrieves the categories of a Wikipedia page using the wikipediaapi package.
    Args: title (str): The title of the Wikipedia page.
    Returns: categories (list): A list of categories associated with the page.
    """
    wikipedia = wikipediaapi.Wikipedia("en")
    page = wikipedia.page(title)
    #print(page)
    if not page.exists():
        return []
    categories = [c for c in page.categories]
    return [cat.split(":")[1] for cat in categories]


In [165]:
def get_category_subcategories(link):
    #print(link)
    # Create a Wikipedia API object
    wiki_api = wikipediaapi.Wikipedia('en')

    # Extract the category name from the link
    category_name = link.split('/')[-1]

    # Retrieve the category page
    category_page = wiki_api.page(f"Category:{category_name}")

    # Find the subcategories section of the page
    subcategories_section = category_page.categorymembers

    # Extract the subcategories from the section
    subcategories = []
    i=0

    for subcategory in subcategories_section.values():
      if i < 100: #threshold for how many entries per category
        i +=1
        if subcategory.ns == wikipediaapi.Namespace.CATEGORY:
            subcategories.append(subcategory.fullurl)

    return subcategories


def get_category_pages(category_title, limit=100):
    params = {
        "action": "query",
        "format": "json",
        "list": "categorymembers",
        "cmtitle": category_title,
        "cmlimit": str(limit) # limit to 100 entries
    }
    #print("OK")
    pages = []
    while True:
        response = requests.get("https://en.wikipedia.org/w/api.php", params=params).json()
        pages += [p["title"] for p in response["query"]["categorymembers"]]
        if "continue" in response:
            params.update(response["continue"])
        else:
            break
        if len(pages) >= limit: # break the loop if the number of entries exceeds 100
            break
    return pages[:limit] # return only the first 100 entries


In [162]:
def get_cat_with_all_subcats(searched_location):
    """
    Given a list of Wikipedia category names, retrieves the number of entries in each category and returns a dictionary
    with the category names as keys and the entry counts as values.
    Args: searched_location: The desired location for wich a hint should be created. (The answer to the question)
    Returns: dicto (dict): A dictionary of all categories with the subcategories and how many entries there are in each.
    """
    #retrieves the list of categories from the location-wikipedia page
    categories = get_wikipedia_categories(searched_location)
    categories_with_underscore = get_category_with_underscores(categories)
    categories_links = get_category_links(categories)

    categories_with_links_dict = {}
    for i in range(len(categories_with_underscore)):
      key = categories_with_underscore[i]
      link = categories_links[i]
      categories_with_links_dict[key] = link

    cat_with_subcats_dict = {}
    for category in categories_links:
      sub_cats = get_category_subcategories(category)
      #print(category)
      ct = get_category_title(category)
      #print(ct)
      pages_list = get_category_pages(ct)
      #print(pages_list)
      filtered_list = [str(entry) for entry in pages_list if not entry.startswith("Category:")]
      new_list = [len(filtered_list), filtered_list]
      if sub_cats is None:
        continue
      else:
        cat_with_subcats_dict[category] = [[len(sub_cats), sub_cats], new_list]

    return cat_with_subcats_dict

#input a url of a category, this returns the tilte
def get_category_title(category_url):
    parts = category_url.split('/')
    title = [part for part in parts if part.startswith('Category:')]
    if title:
        return title[0]
    else:
        return None

#input searched location and returns a dict with number of pages and number of subcategories
def get_categories_ranking(searched_location):
  categories = get_wikipedia_categories(searched_location)
  categories_links = []
  cat_without_articles = []

  bad_list = ['Articles with', 'CS1', 'Wikipedia', 'Webarchive', 'Short', 'Biography', 'Commons', 'Pages', 'Use', 'All', 'Articles', 'Coordinates', 'Engvar', 'Lang', 'Official' ]

  for c in categories:
    if not any(c.startswith(word) for word in bad_list):
        cat_without_articles.append(c)
    
  categories_links = get_category_links(cat_without_articles)
  cat_with_amount = {}

  for category in categories_links:
    sub_cats = get_category_subcategories(category)
    ct = get_category_title(category)
    pages_list = get_category_pages(ct)

    if sub_cats is None:
      continue
    else:
      cat_with_amount[category] =  len(pages_list), len(sub_cats)
  #x[0] sort after subcats and x[1] sorts after pages
  sorted_dict = dict(sorted(cat_with_amount.items(), key=lambda x: x[1], reverse=True))
  
  return sorted_dict

#NEW

#extract the category part from the wiki links
def extract_last_parts(links):
    last_parts = []
    for link in links:
        last_part = link.split('/')[-1]
        last_parts.append(last_part)
    return last_parts

#function to concatenate the category links to insert into a pageviews url
def concatenate_elements(elements, n=10):
    result = []
    for i in range(0, len(elements), n):
        group = elements[i:i+n]
        result.append('|'.join(group))
    return result

"""
This function first initializes an empty list url_list that will hold the modified URLs. Then, it uses a for loop to iterate through each combined string in the input list. 
For each combined string, it concatenates the base URL with a forward slash and the combined string, and appends the resulting URL to the url_list. Finally, the function returns the url_list at the end.
"""
def combine_pv_urls(base_url, combined_strings):
    url_list = []
    for strin in combined_strings:
        url_list.append(base_url  + strin)
    return url_list


"""
This function takes a list of links, opens each link to get the data, discards the header and converts the data into a dictionary using the list_to_dict() function, 
and then updates a combined dictionary with the resulting dictionary from each link. Finally, it returns the combined dictionary.
"""
def combine_dicts_from_links(link_list):
    combined_dict = {}
    for link in link_list:
        header, data = get_table_info(link)
        #print(data)
        link_dict = list_to_dict(data)
        combined_dict.update(link_dict)
    return combined_dict

In [128]:
def add_values_to_links(links_dict, values_dict):
    new_dict = {}
    for link, value in links_dict.items():
        key = link.split(':')[-1]
        if key in values_dict:
            value += (values_dict[key],)
        new_dict[link] = value
    return new_dict

def combine_catnumbers_pvs(ord_dict, norm_dict):
  for key in ord_dict:
    if key in norm_dict:
      ord_dict[key] = ord_dict[key] + (norm_dict[key],)
      print(ord_dict)
  return ord_dict

def add_values_to_linkss(links_dict, values_dict):
  new_dict = {}
  for loc, ord_list in links_dict.items():
    for loc2, ord_list_pv in values_dict.items():
      if loc == loc2:
        new_dict[loc] = combine_catnumbers_pvs(ord_list, ord_list_pv)
        #new_dict[loc] = combine_catnumbers_pvs(ord_list_pv, ord_list)
  return new_dict

#combines the pageviews of the categories of the location together with the sub-categories and pages of those subcategories
def combine_pv_cats(cat_dict, pv_dict):
  tmp = cat_dict

  for key, value in cat_dict.items():
    for key2, value2 in pv_dict.items():
      category_name = key.split('/')[-1]
      new_string = key2.replace(' ', '_')

      if category_name == new_string:
        #print(category_name)
        org_tup = cat_dict[key]
        #new_tup = org_tup + (value2, 0)
        new_tup = org_tup + (value2,)
        tmp[key] = new_tup

        if len(new_tup) != 3:
          print("PROBLEM\n")

  return tmp

#combines the pageviews of the categories of the location together with the sub-categories and pages of those subcategories
def get_dict_for_every_location(cat_ranking, cat_with_pv):
  ret=cat_ranking

  for location, value in cat_ranking.items():
    for location1, value1 in cat_with_pv.items():
      if location == location1:
        tmp = combine_pv_cats(value, value1)
        ret[location] = tmp
  return ret

In [129]:
# person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))

# for subject, question in person_questions_dict.items():
  #print(subject)
  # ranking = get_categories_ranking(subject)
  #categories = get_wikipedia_categories(subject)

#print(ranking)
#print(categories)

In [233]:
def get_categories(subject_dict):
  categories_for_subject_dict= {}
  rankings_for_categories_dict = {}
  #creates a dict with all the dicts of each location with its categories and sub-categories
  for subject, question in subject_dict.items():
    dicto = get_cat_with_all_subcats(subject)
    categories_for_subject_dict[subject] = dicto
    ranking = get_categories_ranking(subject)
    ordict = OrderedDict(ranking)
    rankings_for_categories_dict[subject] = ordict

  return rankings_for_categories_dict

#given a dictionary with all the categories, the function returns the categories in a OrderedDict with the corresponding pageviews for each category
def get_pageviews_for_categories(cat_dict):
  #pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=all-time&pages='
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=last-year&pages='
  
  all_cats_with_pvs = {}

  for subject in cat_dict:
    ord_dict = OrderedDict()
    ordered_dict_sub =  cat_dict[subject]
    links_list = [link for link in ordered_dict_sub.keys()]
    #print(links_list)
    pruned_link_parts_list = extract_last_parts(links_list)
    concat_str_for_links = concatenate_elements(pruned_link_parts_list) #combine up to 10 of these links to create a request to pageview
    pageviews_url_list = combine_pv_urls(pageviews_range_url, concat_str_for_links) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page (REUSED FROM YEARS PART)
    #print(pageviews_url_list)
    categories_with_pageviews =combine_dicts_from_links(pageviews_url_list) #now we called the links and retreieved the pageviews; saved them as a dictionary
    #print(categories_with_pageviews)
    ordered_categories_with_pageviews = sort_dict_desc(categories_with_pageviews) #now the list is ordered in ascending order
    #all_cats_with_pvs[loc] = OrderedDict(ordered_categories_with_pageviews)
    all_cats_with_pvs[subject] = OrderedDict(ordered_categories_with_pageviews)
  return all_cats_with_pvs

def sorting_dict(sor_dict):
  ret = {}
  l2 = sor_dict
  #sorting the dict after pageviews
  for loc, value in sor_dict.items():
    for link,tup in value.items():#t1
      if len(tup) < 3:#
        #del value[link]#
        del l2[loc][link]


  for loc, value in l2.items():
    try:
      sorted_dict = OrderedDict(sorted(value.items(), key=lambda x: x[1][2], reverse=True))
    except Exception as e:
      print(f"Sorting failed for location {loc}: {e}")
      sorted_dict = value
    ret[loc] = sorted_dict
    #sorted_dict = dict(sorted(value.items(), key=lambda x: x[2], reverse=True))
  return ret

from collections import OrderedDict

# function that takes a dictionary with an ordered dictionary as the value and prunes the ordered dictionary to keep only the first n entries:
def prune_ordered_dict(dictionary, n):
    pruned_dict = OrderedDict()
    for key, value in dictionary.items():
        pruned_dict[key] = OrderedDict(list(value.items())[:n])
    return pruned_dict


# function that takes a dictionary with an ordered dictionary as the value and prunes the ordered dictionary to keep only the first n entries and deltes certain categories:
def prune_and_ordered_dict(dictionary, n):
  pruned_dict = OrderedDict()
  inter1_dict= OrderedDict()
  for key, value in dictionary.items():
    inter3_dict= OrderedDict()
    for link, tuplee in value.items():
      link_str = str(link)
      if 'Living_people' not in link_str and '_births' not in link_str and '_deaths' not in link_str: 
        inter3_dict[link] = tuplee
    pruned_dict[key] = inter3_dict

  for key, value in pruned_dict.items():
    inter1_dict[key] = OrderedDict(list(value.items())[:n])
  return inter1_dict


In [137]:
#find the 20 most appearing categories 
def find_most_common_links(data_dict):
    link_count = {}
    for key, value in data_dict.items():
        for link in value:
            if link not in link_count:
                link_count[link] = {"count": 1, "keys": [key]}
            else:
                link_count[link]["count"] += 1
                link_count[link]["keys"].append(key)

    # Sort the links by their count in descending order
    sorted_links = sorted(link_count.items(), key=lambda x: x[1]["count"], reverse=True)

    # Return a list of tuples with the link, count, and keys
    return [(link, data["count"], data["keys"]) for link, data in sorted_links[:20]]



## Test it!

In [132]:
#for locations

location_questions_dict = dict(zip(location_df['Answer'], location_df['Question']))
cat_with_pv_location = {}

#for locations
cat_ranking_location = get_categories(location_questions_dict)
#pprint.pprint(cat_ranking_location, indent=1)
cat_with_pv_location = get_pageviews_for_categories(cat_ranking_location)
#pprint.pprint(cat_with_pv_location, indent=1)
#for locations: sorting the dict after pages per category
categories_with_subs_and_pageviews_location = get_dict_for_every_location(cat_ranking_location, cat_with_pv_location)
#pprint.pprint(categories_with_subs_and_pageviews_location,indent=1)
new_ordered_dict_location = sorting_dict(categories_with_subs_and_pageviews_location)
#pprint.pprint(new_ordered_dict_location,indent=1)

#for 3 locations 1m33s

In [166]:
#for person
person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
cat_with_pv_person = {}

print(person_questions_dict)
#del str

#for person
cat_ranking_person = get_categories(person_questions_dict)
#pprint.pprint(cat_ranking_person, indent=1)
cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
#pprint.pprint(cat_with_pv_person, indent=1)
#for person: sorting the dict after pages per category
categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
#pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)
#pprint.pprint(new_ordered_dict_person,indent=1)

#for 3 people 17m22s
#27s
# for 20 F1 drivers -> 20mins

{'Max Verstappen': 'Who was the F1 World Champion in 2022?', 'Michael Jackson': 'Who is the artist behind the song Thriller?', 'Joe Biden': 'Who is the current president of the US?'}
Sorting failed for location Max Verstappen: tuple index out of range
Sorting failed for location Michael Jackson: tuple index out of range
Sorting failed for location Joe Biden: tuple index out of range


In [204]:
def get_categories_with_pageviews_person(person_questions_dict):
  cat_ranking_person = get_categories(person_questions_dict)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  #for person: sorting the dict after pages per category
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  #pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)
  return new_ordered_dict_person

## Print it!

In [167]:
#just for easy printing such that you dont have to0 calculate everything everytime
copy_categories_with_subs_and_pageviews_location = categories_with_subs_and_pageviews_location
copy_new_ordered_dict_location = new_ordered_dict_location
#print("Ordered after the number of pages per category:")
#pprint.pprint(copy_categories_with_subs_and_pageviews_location,indent=1)
print("Ordered after the number of pageviews per category:")
pprint.pprint(copy_new_ordered_dict_location,indent=1, compact=True)

copy_categories_with_subs_and_pageviews_person = categories_with_subs_and_pageviews_person
copy_new_ordered_dict_person = new_ordered_dict_person
#print("Ordered after the number of pages per category:")
#pprint.pprint(copy_categories_with_subs_and_pageviews_person,indent=1)
print("Ordered after the number of pageviews per category:")
#prune the dict down to 20 most visited categories per person
#copy_new_ordered_dict_person = prune_ordered_dict(copy_new_ordered_dict_person, 20)
#first delete categories like (livingpeople, deaths, births) and prune the dict down to 20 most visited categories per person
copy_new_ordered_dict_person_test = prune_and_ordered_dict(copy_new_ordered_dict_person, 20)
copy_new_ordered_dict_person = sorting_dict(copy_new_ordered_dict_person_test)
pprint.pprint(copy_new_ordered_dict_person,indent=1, compact=True)

#8.30 for 10 drivers
category_occurences = find_most_common_links(copy_new_ordered_dict_person)
print('These are the categories that occur most between the person-entities:')
pprint.pprint(category_occurences, indent=1, compact=True)

Ordered after the number of pageviews per category:
{}
Ordered after the number of pageviews per category:
Sorting failed for location Max Verstappen: tuple index out of range
{'Joe Biden': OrderedDict([('https://en.wikipedia.org/wiki/Category:American_people_of_Irish_descent',
                            (100, 0, 32705)),
                           ('https://en.wikipedia.org/wiki/Category:American_people_of_English_descent',
                            (100, 0, 22035)),
                           ('https://en.wikipedia.org/wiki/Category:Presidents_of_the_United_States',
                            (100, 0, 18140)),
                           ('https://en.wikipedia.org/wiki/Category:American_Roman_Catholics',
                            (100, 0, 14103)),
                           ('https://en.wikipedia.org/wiki/Category:Presidential_Medal_of_Freedom_recipients',
                            (100, 0, 10987)),
                           ('https://en.wikipedia.org/wiki/Category:20th-centu

In [169]:
#create the hints with the categories
#at the moment the program is just working with person/location that have english wiki pages
#create the actual hint-sentences

print("Generated by the ordering of how many pages there are in a category:")
pprint.pprint(get_first_three_categs_location(copy_categories_with_subs_and_pageviews_location))
print("Generated by the ordering of how many pageviews a category has:")
pprint.pprint(get_first_three_categs_location(new_ordered_dict_location))

print("Generated by the ordering of how many pages there are in a category:")
pprint.pprint(get_first_three_categs_person(copy_categories_with_subs_and_pageviews_person))
print("Generated by the ordering of how many pageviews a category has:")
pprint.pprint(get_first_three_categs_person(new_ordered_dict_person))

Generated by the ordering of how many pages there are in a category:
{}
Generated by the ordering of how many pageviews a category has:
{}
Generated by the ordering of how many pages there are in a category:
{'Joe Biden': ['The person you are looking for, belongs to the category 1942 '
               'births',
               'The person you are looking for, belongs to the category '
               '20th-century American lawyers',
               'The person you are looking for, belongs to the category '
               '20th-century American politicians'],
 'Max Verstappen': ['The person you are looking for, belongs to the category '
                    '1997 births',
                    'The person you are looking for, belongs to the category '
                    'Belgian racing drivers',
                    'The person you are looking for, belongs to the category '
                    'Dutch racing drivers'],
 'Michael Jackson': ['The person you are looking for, belongs to the categor

## Testing/Calculating unexpectedness via categories

In [172]:
import itertools
import requests

template_sentence_location = 'The location you are looking for, is a member of the category x'
template_sentence_location2 = 'The location you are looking for, belongs to the category '

template_sentence_person = 'The person you are looking for, is a member of the category x'
template_sentence_person2 = 'The person you are looking for, belongs to the category '

#retrives the category name from the link
def get_category_name(link):
    category_prefix = "Category:"
    if link.startswith("https://wikipedia.org/wiki/"):
        return link[len("https://wikipedia.org/wiki/" + category_prefix):].replace("_", " ")
    else:
      if link.startswith("https://en.wikipedia.org/wiki/"):
        return link[len("https://en.wikipedia.org/wiki/" + category_prefix):].replace("_", " ")
      else:
        return None
        
#just get the first 3 categories without any special formula
def get_first_three_categs_location(copy_categories_with_subs_and_pageviews_location):
  first_three_categories_per_location = {}
  for key, value in copy_categories_with_subs_and_pageviews_location.items():
    first_three = dict(itertools.islice(value.items(), 3))
    first_three_categories_per_location[key] = first_three

  hint_sentence_location = {}
  for key, value in first_three_categories_per_location.items():
    inter_hints = []
    for link, tup in value.items():
      cat_name=get_category_name(link)
      if cat_name is not None:
        ok = template_sentence_location2 + cat_name
      else: 
        ok = template_sentence_location2
      inter_hints.append(ok)
    hint_sentence_location[key] = inter_hints
  return hint_sentence_location

#just get the first 3 categories without any special formula
def get_first_three_categs_person(copy_categories_with_subs_and_pageviews_location):
  first_three_categories_per_location = {}
  for key, value in copy_categories_with_subs_and_pageviews_location.items():
    first_three = dict(itertools.islice(value.items(), 3))
    first_three_categories_per_location[key] = first_three

  hint_sentence_location = {}
  for key, value in first_three_categories_per_location.items():
    inter_hints = []
    for link, tup in value.items():
      cat_name=get_category_name(link)
      if cat_name is not None:
        ok = template_sentence_person2 + cat_name
      else: 
        ok = template_sentence_person2
      inter_hints.append(ok)
    hint_sentence_location[key] = inter_hints
  return hint_sentence_location

#get predicates of a wiki page via wikidataAPI
def get_wikidata_predicates(page_title):
    # First, get the Wikidata item ID of the page
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles={page_title}&format=json"
    response = requests.get(url).json()
    pages = response["query"]["pages"]
    if "-1" in pages:
        return None
    page_id = next(iter(pages))
    wikidata_id = pages[page_id]["pageprops"]["wikibase_item"]

    # Then, get the predicates and their values of the Wikidata item
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={wikidata_id}&format=json&props=claims"
    response = requests.get(url).json()
    entity = response["entities"][wikidata_id]
    predicates = {}
    for claim_id, claim in entity["claims"].items():
        predicate_id = claim["mainsnak"]["property"]
        predicate_value = claim["mainsnak"]["datavalue"]["value"]
        predicates[predicate_id] = predicate_value
    return predicates

#
#TEST
#

people_list=[]
for l in dataPerson:
  people_list.append(l[1])

# takes a list of all the people and the list of category occurences where all the categories that were assigned to these wiki-pages 
# are listed and ranked after how often they occur; 
# output: a list where the person is the key and the entries show wich categories the entity shares with wich other entitities and how many they are
def get_people_dict(people_list, occurences_list):
    people_dict = {}
    for person in people_list:
        person_categories = []
        for occurence in occurences_list:
            if person in occurence[2]:
                person_categories.append((occurence[0], occurence[1], occurence[2]))
        people_dict[person] = person_categories
    return people_dict

#returns a dict where for each person in people_list we calculate how often they occur in the same category as the keys in person_dict 
# (where already for each person we looked at each of the categories he appears and counted how many other people from people_list appear in these )
def count_people_occurrences(person_dict, people_list):
  # Initialize an empty dictionary to hold the counts  
  count_dict = {}
  # Loop through all pairs of drivers and count the number of occurrences
  for key, value in person_dict.items():
    counter = {}
    for person in people_list:
      counter[person] = 0
      for cat in value:
        if person in cat[2]:
          counter[person] += 1  
    count_dict[key] = counter
  return count_dict

#sorting
def sort_dict_by_value_desc(d):
    # Sort the inner dictionary by value in descending order
    sorted_dict = OrderedDict(sorted(d.items(), key=lambda x: x[1], reverse=True))
    return sorted_dict

def get_categories_union(overlap_dict, people_list):
  # Initialize an empty dictionary to hold the counts  
  count_dict = {}
  for key, value in overlap_dict.items():
    counter = {}
    for item, number in value.items():
      #print(number)
      counter[item] = number_categories_per_person[key] + number_categories_per_person[item] - number
    count_dict[key] = counter
  return count_dict

def get_avg_pairwise_sim(cat_div_union, cat_div_overlap, number_categories_per_person, people_list):
  count_dict = {}
  for key, value in cat_div_union.items():
    inter = {}
    for key1, value1 in cat_div_overlap.items():
      if key == key1:
        for name, number in value.items():
          for name1, number1 in value1.items():
            if name == name1:
              if not number  or not number1:
                inter[name] = 0
              else:
                inter[name] = (number1 / number)
    count_dict[key] = inter
  return count_dict

def get_cat_diversity(shared_categories, copy_new_ordered_dict_person):
  count_dict = {}
  for key, value in copy_new_ordered_dict_person.items():
    for item in value.items():
      pv = 0
      #print(item)
      link = item[0]
      trip = item[1]
      if len(trip) == 3:
        pv = trip[2]
      if link not in count_dict:
        count_dict[link] = pv
  return count_dict

In [173]:
#create scores and metric to determine the best hint

#The unexpectedness approach I am taking at the moment is:
#1. Look up the occupation of the person-answer-entity
#2. Search for 3-5 similar/comparable people from the same occupation (direction)
#3, Gather the 20 most popular categories from all entities (rank them)
#4. Try to find a popular category (piece of information) that is untypical for most people from this occupation or field-of-work

#category diversity = avg pairwise-similarity of members (=Jaccard similarity coefficient = IoU)
#IoU = intersection over union = (#shared categories of A and B) / (#categories of A + #categories of B)


#wich drivers shares what category with wich other driver
shared_categories = get_people_dict(people_list, category_occurences)
pprint.pprint(shared_categories, indent=1, compact=True)

print("People dict with the number of how many categories they appear in:")
number_categories_per_person = {}
for key, value in copy_new_ordered_dict_person.items():
    number_categories_per_person[key] = len(value)
#pprint.pprint(number_categories_per_person, indent=1, compact=True)
print("\n")

print("Category overlap: how many categories in A AND B together")
unsorted_category_diversity = count_people_occurrences(shared_categories, people_list)
cat_div_overlap = {k: sort_dict_by_value_desc(v) for k, v in unsorted_category_diversity.items()}
#pprint.pprint(cat_div_overlap, indent=1, compact=True)
print("\n")

print("Category union: how many categories in A OR B")
unsorted_cat_div_union = get_categories_union(cat_div_overlap, people_list)
cat_div_union = {k: sort_dict_by_value_desc(v) for k, v in unsorted_cat_div_union.items()}
#pprint.pprint(cat_div_union, indent=1, compact=True)
print("\n")

#avg pairwise-similarity of each person to each other person in people_list
print("Cat_diversity = avg_pairwise_similarity = IoU")
unordered_avg_pairwise_sim = get_avg_pairwise_sim(cat_div_union, cat_div_overlap, number_categories_per_person, people_list)
avg_pairwise_sim = {k: sort_dict_by_value_desc(v) for k, v in unordered_avg_pairwise_sim.items()}
pprint.pprint(avg_pairwise_sim, indent=1, compact=True)


#avg pairwise-similarity of each ossuring category in the wiki pages of the people
# cat_popularity = avg view count of categories that appear in the persons wiki pages ordered descending after the highest
print("Cat_popularity = avg view count of category")
unordered_cat_popularity= get_cat_diversity(category_occurences, copy_new_ordered_dict_person)
cat_popularity = OrderedDict(sorted(unordered_cat_popularity.items(), key=lambda x: x[1], reverse=True))
pprint.pprint(cat_popularity, indent=1, compact=True)

#1. write a function that finds similar entities to our searched entity (3-5 comparable entities)
#2. compare those via avg pairwise to 


{'Joe Biden': [],
 'Max Verstappen': [('https://en.wikipedia.org/wiki/Category:Belgian_racing_drivers',
                     1, ['Max Verstappen']),
                    ('https://en.wikipedia.org/wiki/Category:Dutch_racing_drivers',
                     1, ['Max Verstappen']),
                    ('https://en.wikipedia.org/wiki/Category:FIA_Formula_3_European_Championship_drivers',
                     1, ['Max Verstappen']),
                    ('https://en.wikipedia.org/wiki/Category:Formula_One_race_winners',
                     1, ['Max Verstappen']),
                    ('https://en.wikipedia.org/wiki/Category:Karting_World_Championship_drivers',
                     1, ['Max Verstappen']),
                    ('https://en.wikipedia.org/wiki/Category:Motopark_Academy_drivers',
                     1, ['Max Verstappen']),
                    ('https://en.wikipedia.org/wiki/Category:Officers_of_the_Order_of_Orange-Nassau',
                     1, ['Max Verstappen']),
              

In [174]:
def get_categories_with_ranking():
  #for person
  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
  cat_with_pv_person = {}

  #for person
  cat_ranking_person = get_categories(person_questions_dict)
  #pprint.pprint(cat_ranking_person, indent=1)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  #pprint.pprint(cat_with_pv_person, indent=1)
  #for person: sorting the dict after pages per category
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  #pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)
  #pprint.pprint(new_ordered_dict_person,indent=1)

  return new_ordered_dict_person

all_people_cat_ranked = get_categories_with_ranking() #copy_new_ordered_dict_person

Sorting failed for location Max Verstappen: tuple index out of range
Sorting failed for location Michael Jackson: tuple index out of range
Sorting failed for location Joe Biden: tuple index out of range


In [175]:

import wikipediaapi
import requests

#given a name, retrieve the infomration in the short-description part of the wiki page
def get_page_short_description(page_title):
    # Prepare the API request URL
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title.replace(' ', '_')}"

    # Send the API request
    response = requests.get(url)
    data = response.json()

    # Extract the short description from the API response
    short_description = data.get('description', '')

    return short_description

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import OrderedDict

def find_most_similar_category(persons, specific_category, num_similar_categories=3):
    most_similar_categories = {}

    for person, categories in persons.items():
        category_texts = [category_link.split('/')[-1].replace('_', ' ') for category_link in categories.keys()]
        category_texts.append(specific_category[person])  # Add the specific category for comparison

        # Vectorize the category texts
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(category_texts)
        #print(category_texts)
        # Calculate cosine similarity between the specific category and other categories
        similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
         # Calculate cosine similarity between the specific category and other categories
        similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]
        # Find the most similar categories textually
        similar_indices = similarities.argsort()[-num_similar_categories:][::-1]
        similar_categories = [category_texts[index] for index in similar_indices]
        most_similar_categories[person] = similar_categories
    return most_similar_categories


def get_work_category(similar_categories):
  person_with_work_categories = {}
  person_with_work_category = {}
  strings_to_check = ["births", "deaths"]
  for person, categories in similar_categories.items():
    for strs in strings_to_check:
      for cats in categories:
        #print(person , strs)
        if strs in cats:
          #print(cats)
          categories.remove(cats)
        else:
          continue
      person_with_work_categories[person] = categories
  for person, categories in person_with_work_categories.items():
    person_with_work_category[person] = categories[0]
  return person_with_work_category



def get_container_categories(work_cats):
  for person, categories in work_cats.items():
    wiki_category = categories
    base_url = 'https://en.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'titles': wiki_category,
        'prop': 'categories',
        'format': 'json',
        'cllimit': 'max'
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    page_id = next(iter(data['query']['pages'].keys()))
    categories = data['query']['pages'][page_id].get('categories', [])
    container_categories = []
    for category in categories:
        if category['title'].startswith('Category:'):
            container_categories.append(category['title'][9:])

    strings_to_check = ["CatAutoTOC ", "Commons", "Template"]
    for strs in strings_to_check:
      for cats in container_categories:
        if strs in cats:
          container_categories.remove(cats)
        else:
          continue
    work_cats[person] = container_categories
  return work_cats

def replace_spaces_with_underscore(dictionary):
    updated_dict = {}
    for key, value in dictionary.items():
        updated_dict[key] = value.replace(" ", "_")
    return updated_dict

def format_category_dict(category_dict):
    formatted_dict = {}
    for key, value in category_dict.items():
        if isinstance(value, list):
            formatted_value = [f"Category:{v.replace(' ', '_')}" for v in value]
        else:
            formatted_value = f"Category:{value.replace(' ', '_')}"
        formatted_dict[key] = formatted_value
    return formatted_dict



#dict that contains the person as key and the occupation as value
person_with_job = {}
for person in people_list:
  person_with_job[person] = get_page_short_description(person) 

#this is a dict where the person name is the key and the value is the category of the work place
#print(person_with_job)
print(all_people_cat_ranked)

#this is a dict where the person name is the key and a list of 3 most similar categories to the "job-category" of the person
similar_categories = find_most_similar_category(all_people_cat_ranked, person_with_job)
print(similar_categories)

#return the work category of the person
work_category = get_work_category(similar_categories)
print("wk " , work_category)

#get the container category
container_category_work = get_container_categories(work_category)
print(container_category_work)

#format the dictionary
test2 = format_category_dict(container_category_work)
print(test2)



#test = get_container_categories(format_category_dict(wk))
#print(test)


#print(get_container_categories(test))

#test3 = get_pageviews_for_categories(test2)
#print(test3)

#test4 = get_dict_for_every_location(test2, cat_with_pv_location)

#Nowthat we have the parent categories, lets go one step above to get Racing drivers instead of nationality of racing drivers
# now look for

{'Max Verstappen': OrderedDict([('https://en.wikipedia.org/wiki/Category:1997_births', (100, 0, 21263)), ('https://en.wikipedia.org/wiki/Category:Belgian_racing_drivers', (100, 0, 1271)), ('https://en.wikipedia.org/wiki/Category:Dutch_racing_drivers', (100, 0, 1714)), ('https://en.wikipedia.org/wiki/Category:FIA_Formula_3_European_Championship_drivers', (100, 0, 163)), ('https://en.wikipedia.org/wiki/Category:Formula_One_race_winners', (100, 0, 1111)), ('https://en.wikipedia.org/wiki/Category:Karting_World_Championship_drivers', (100, 0, 808)), ('https://en.wikipedia.org/wiki/Category:Living_people', (100, 0, 509392)), ('https://en.wikipedia.org/wiki/Category:Motopark_Academy_drivers', (100, 0, 140)), ('https://en.wikipedia.org/wiki/Category:Officers_of_the_Order_of_Orange-Nassau', (100, 0, 3055)), ('https://en.wikipedia.org/wiki/Category:Twitch_(service)_streamers', (100, 0, 14054)), ('https://en.wikipedia.org/wiki/Category:Van_Amersfoort_Racing_drivers', (97, 0, 119)), ('https://en.w

In [176]:
import requests
from bs4 import BeautifulSoup

#The wikipedia-api library does not provide a direct method to retrieve the infobox of a Wikipedia page. 
#However, we can use beautifulsoup4 to parse the HTML content of the Wikipedia page and extract the infobox.
#function that retrieves the infobox using beautifulsoup4
def get_infobox_from_wikipedia(page_title):
    # Format the page title for the Wikipedia URL
    formatted_title = page_title.replace(' ', '_')
    # Construct the Wikipedia page URL
    url = f"https://en.wikipedia.org/wiki/{formatted_title}"
    # Send a GET request to the Wikipedia page
    response = requests.get(url)
    # Check if the page exists
    if response.status_code != 200:
        print(f"Page '{page_title}' does not exist.")
        return None
    # Create a BeautifulSoup object to parse the page content
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the infobox section of the page
    infobox = soup.find(class_='infobox')
    return infobox

#function that retrieves the infobox from a Wikipedia page and specifically extracts the entries from the "occupation" or "occupations" field:
def get_occupations_from_infobox(page_title):
    # Retrieve the infobox from the Wikipedia page
    infobox = get_infobox_from_wikipedia(page_title)
    # Check if the infobox exists
    if not infobox:
        return []
    # Search for the "occupation" or "occupations" field in the infobox
    occupation_keywords = ['occupation', 'occupations', 'Occupation', 'Occupations']
    occupations = []
    for keyword in occupation_keywords:
        field = infobox.find(string=keyword)
        if field:
            # Retrieve the occupation entries from the field
            tmp_str = field.find_next("td").find_next("div").find_next("ul")
            from_table = extract_list_elements(tmp_str)
            from_field = field.find_next("td").text.strip()
            from_field = from_field.split()
            if len(from_table) > 0:
              for i in from_table:
                occupations.append(i)
            elif len(from_field) > 0:
              for i in from_field:
                occupations.append(i)
            else:
              print("prpoblem")
    return occupations

#extract the elements between the <li> tags
def extract_list_elements(html_string):
  occupations_list = []
  li_entries_list = []
  html_str = str(html_string)
  html_str = html_str.replace("</ul>", "")
  html_str = html_str.replace("<ul>", "")
  html_str = html_str.replace("</li>", "")
  li_entries_list = html_str.split("<li>")
  for i in li_entries_list:
    if len(i) > 0: #check if empty
      if len(i) < 16: #check if entry is a link
        occupations_list.append(i)
  return occupations_list

#searches the occupation for every entry in people list
def get_occupations(people_list):
  occupation_person_dict = {}
  for person in people_list:
    occupation_person_dict[person] = get_occupations_from_infobox(person) 
  return occupation_person_dict

#TEST IT OUT?
people_list=[]
for l in dataPerson:
  people_list.append(l[1])

occupation_person_dict = get_occupations(people_list)
print(occupation_person_dict)


{'Max Verstappen': ['Racing', 'driver'], 'Michael Jackson': ['Singer', 'songwriter', 'dancer', 'record producer'], 'Joe Biden': ['Politician', 'lawyer', 'author']}


In [216]:
"""
1. retrieve all of the related links of a wiki-persons-page (first 500);
2. pre prune the list, such that only entries with 2 words in the title are left; (for performance reasons)
3. analyse the links that are left via wikipedia APi and the "instance of" (P31) property set to "human" (Q5)
4. now we have a list of related people to compare to our answer-person-entity
"""
import random

#retrieves all of the relate links of a wiki page
def get_related_links(wiki_link):
    # Extract the page title from the Wikipedia link
    title = wiki_link.split('/')[-1]
    # Format the API URL to get the page content
    api_url = f'https://en.wikipedia.org/w/api.php?action=query&titles={title}&prop=links&pllimit=max&format=json'
    # Send a GET request to the API
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        # Extract the page ID from the API response
        page_id = next(iter(data['query']['pages']))
        # Check if the page exists and has links
        if page_id != '-1' and 'links' in data['query']['pages'][page_id]:
            # Retrieve the links from the API response
            links = data['query']['pages'][page_id]['links']
            # Extract the link titles and URLs
            related_links = [{'url': f'https://en.wikipedia.org/wiki/{link["title"]}', 'title': link['title']} for link in links]
            return related_links
    return []

# function to check if the title consists of two words -> discrad a lot of entries for performance reasons
def filter_two_word_titles(links):
    filtered_links = []
    url_concat = ""
    for link in links:
      url = link['url']
      link['url'] = url.replace(' ', '_')
      title = link['title']
      words = title.split()
      if len(words) == 2:
        filtered_links.append(link)

    return filtered_links

#check if a given Wikipedia link corresponds to an entity with the "instance of" (P31) property set to "human" (Q5), indicating it represents a person
def is_person_page(link):
    # Extract the page title from the link
    title = link['title']
    # Query Wikidata API to check if the page corresponds to a person
    wikidata_url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&sites=enwiki&titles={title}"
    response = requests.get(wikidata_url).json()
    # Check if the response contains any entities
    if 'entities' in response:
        entities = response['entities']
        # Check if there is an entity with P31 (instance of) set to Q5 (human)
        for entity_id, entity in entities.items():
            if 'claims' in entity and 'P31' in entity['claims']:
                for claim in entity['claims']['P31']:
                    if claim['mainsnak']['datavalue']['value']['id'] == 'Q5':
                        return True
    return False

#function that calls the is_perso_page() function for every link in the pre discraded list
def check_if_person(links):
  person_links= []
  for link in links:
    if is_person_page(link):
       person_links.append(link)
  return person_links

#function that selects 5 entries at random from a list of related people:
def select_random_entries(related_people_list, num_entries=5):
    random_entries = random.sample(related_people_list, num_entries)
    return random_entries


#TEST
#


# takes a list of all the people and the list of category occurences where all the categories that were assigned to these wiki-pages 
# are listed and ranked after how often they occur; 
# output: a list where the person is the key and the entries show wich categories the entity shares with wich other entitities and how many they are
def get_people_dict(people_list, occurences_list):
    people_dict = {}
    for person in people_list:
        person_categories = []
        for occurence in occurences_list:
            if person in occurence[2]:
                person_categories.append((occurence[0], occurence[1], occurence[2]))
        people_dict[person] = person_categories
    return people_dict
##
##

#
#
wiki_link = 'https://en.wikipedia.org/wiki/Max_Verstappen'
related_articles = get_related_links(wiki_link)
print("related articles:",  len(related_articles))

filtered_links = filter_two_word_titles(related_articles)
print("filtered articles:",  len(filtered_links))
#print(filtered_links)

related_people_list = check_if_person(filtered_links)
print("related_people_list:",  len(related_people_list))
print(related_people_list)

select_random_people = select_random_entries(related_people_list)
print("select_random_people:",  len(select_random_people))

pprint.pprint(select_random_people)

#given a dictionary with all the categories, the function returns the categories in a OrderedDict with the corresponding pageviews for each category
#for k,v in related_people_list.items():
#nice =  get_pageviews_for_categories(related_people_list):

def get_most_known_related_people(related_people_list):
  pageviews_range_url = 'https://pageviews.wmcloud.org/?project=en.wikipedia.org&platform=all-access&agent=user&redirects=0&range=last-year&pages='
  ord_dict = OrderedDict()
  link_list = []
  for item in related_people_list:
    link_list.append(item['url'])
  pruned_link_parts_list = extract_last_parts(link_list)
  concat_str_for_links = concatenate_elements(pruned_link_parts_list) #combine up to 10 of these links to create a request to pageview
  pageviews_url_list = combine_pv_urls(pageviews_range_url, concat_str_for_links) #now we have a list of pageview links with all the backlinks of the thumbcaption part of the wiki page (REUSED FROM YEARS PART)
  categories_with_pageviews =combine_dicts_from_links(pageviews_url_list) #now we called the links and retreieved the pageviews; saved them as a dictionary
  ordered_categories_with_pageviews = sort_dict_desc(categories_with_pageviews) #now the list is ordered in ascending order
  ord_dict = ordered_categories_with_pageviews

  #print(ordered_categories_with_pageviews)
  return ord_dict

#TEST
#prunedAndOrdered = prune_and_ordered_dict(copy_new_ordered_dict_person, 20)
#pprint.pprint(prunedAndOrdered)

# #for person
# cat_ranking_person = get_categories(person_questions_dict)
# #pprint.pprint(cat_ranking_person, indent=1)
# cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
# #pprint.pprint(cat_with_pv_person, indent=1)
# #for person: sorting the dict after pages per category
# categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
# #pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
# new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)
# #pprint.pprint(new_ordered_dict_person,indent=1)


#people_list=[]
#for l in dataPerson:
#  people_list.append(l[1])

#shared_categories = get_people_dict(people_list, category_occurences)
#pprint.pprint(shared_categories, indent=1, compact=True)

tester = get_most_known_related_people(related_people_list)
print(tester)
pprint.pprint(tester)



related articles: 500
filtered articles: 132
related_people_list: 101
[{'url': 'https://en.wikipedia.org/wiki/Adrian_Newey', 'title': 'Adrian Newey'}, {'url': 'https://en.wikipedia.org/wiki/Alain_Prost', 'title': 'Alain Prost'}, {'url': 'https://en.wikipedia.org/wiki/Alberto_Ascari', 'title': 'Alberto Ascari'}, {'url': 'https://en.wikipedia.org/wiki/Alex_Albon', 'title': 'Alex Albon'}, {'url': 'https://en.wikipedia.org/wiki/Alex_Hitzinger', 'title': 'Alex Hitzinger'}, {'url': 'https://en.wikipedia.org/wiki/Alex_Zanardi', 'title': 'Alex Zanardi'}, {'url': 'https://en.wikipedia.org/wiki/Alexandre_Prémat', 'title': 'Alexandre Prémat'}, {'url': 'https://en.wikipedia.org/wiki/Andrew_Shovlin', 'title': 'Andrew Shovlin'}, {'url': 'https://en.wikipedia.org/wiki/Anthony_Kumpen', 'title': 'Anthony Kumpen'}, {'url': 'https://en.wikipedia.org/wiki/Antonio_Fuoco', 'title': 'Antonio Fuoco'}, {'url': 'https://en.wikipedia.org/wiki/Antonio_Giovinazzi', 'title': 'Antonio Giovinazzi'}, {'url': 'https://

In [242]:
#Test out new way:
#list ordered after most well-known related people to our answer-entity
# take 5 most well known people and calculate the categories with pageviews for all of them
#  take 20 most popular categories and compare those to the other related-people and the answer-entity
#  rank after the categories that appear most
#  rank categories that appear only for our answer-entity
#   choose the most popular one for the hints (most-popular category, in wich our answer-entity is the only one of the related-people that appears in)

#tester = get_most_known_related_people(related_people_list)

related_people_ranked = dict(sorted(tester.items(), key=lambda x: x[1], reverse=True))
top_most_popular_people = dict(itertools.islice(related_people_ranked.items(), 5))#take the top 5 most known people from the list
#for key, value in top_most_popular_people.items():
cat_ranking_related_person = get_categories(top_most_popular_people)
print(cat_ranking_related_person)
cat_with_pv_person = get_pageviews_for_categories(cat_ranking_related_person)
print(cat_with_pv_person)
#categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_related_person, cat_with_pv_person)
print(categories_with_subs_and_pageviews_person)


new_ordered_dict_related_person = sorting_dict(categories_with_subs_and_pageviews_person)

copy_new_ordered_dict_person_test = prune_and_ordered_dict(new_ordered_dict_related_person, 20)
ordered_dict_related_person = sorting_dict(copy_new_ordered_dict_person_test)

pprint.pprint(OrderedDict(ordered_dict_related_person)) 
#pprint.pprint(OrderedDict(related_people_ranked))
#5:04

{'Lewis Hamilton': OrderedDict([('https://en.wikipedia.org/wiki/Category:1985_births', (100, 0)), ('https://en.wikipedia.org/wiki/Category:ART_Grand_Prix_drivers', (100, 0)), ('https://en.wikipedia.org/wiki/Category:Black_British_sportsmen', (100, 0)), ('https://en.wikipedia.org/wiki/Category:British_Formula_Renault_2.0_drivers', (100, 0)), ('https://en.wikipedia.org/wiki/Category:English_Formula_One_drivers', (100, 0)), ('https://en.wikipedia.org/wiki/Category:English_Roman_Catholics', (100, 0)), ('https://en.wikipedia.org/wiki/Category:English_philanthropists', (100, 0)), ('https://en.wikipedia.org/wiki/Category:English_racing_drivers', (100, 0)), ('https://en.wikipedia.org/wiki/Category:Formula_3_Euro_Series_drivers', (100, 0)), ('https://en.wikipedia.org/wiki/Category:Formula_One_race_winners', (100, 0)), ('https://en.wikipedia.org/wiki/Category:Formula_Renault_Eurocup_drivers', (100, 0)), ('https://en.wikipedia.org/wiki/Category:GP2_Series_drivers', (100, 0)), ('https://en.wikiped

In [256]:
#copy = ordered_dict_related_person
#pprint.pprint(OrderedDict(get_ordered_categories_of_most_related_people())) 


#IMPORTANT
def get_ordered_categories_of_most_related_people(most_related_peoples_list):
#  tester = get_most_known_related_people()
  ret = {}

  for k,v in most_related_peoples_list.items():
    related_people_ranked = dict(sorted(v.items(), key=lambda x: x[1], reverse=True))
    top_most_popular_people = dict(itertools.islice(related_people_ranked.items(), 5))#take the top 5 most known people from the list
    #for key, value in top_most_popular_people.items():
    cat_ranking_related_person = get_categories(top_most_popular_people)
    print(cat_ranking_related_person)
    cat_with_pv_person = get_pageviews_for_categories(cat_ranking_related_person)
    print(cat_with_pv_person)
    #categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
    categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_related_person, cat_with_pv_person)
    print(categories_with_subs_and_pageviews_person)

    new_ordered_dict_related_person = sorting_dict(categories_with_subs_and_pageviews_person)

    copy_new_ordered_dict_person_test = prune_and_ordered_dict(new_ordered_dict_related_person, 20)
    ordered_dict_related_person = sorting_dict(copy_new_ordered_dict_person_test)
    ret[key] = ordered_dict_related_person
  return ret


#IMPORTANT
#get the links of all answer-entities via link creation of the key
def get_rel_pep():
  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
  wiki_link = 'https://en.wikipedia.org/wiki/'
  ret = {}

  for key,value in person_questions_dict.items():
    #ret
    modified_text = key.replace(' ', '_')
    link = f"{wiki_link}/{modified_text}" 
    related_articles = get_related_links(link)
    filtered_links = filter_two_word_titles(related_articles)
    related_people_list = check_if_person(filtered_links)
    ret[key] = related_people_list
  return ret

a = get_rel_pep()
print(a)

inter = {}

#for all of the related people, search the pageviews
for k,v in a.items():
  inter[k] = get_most_known_related_people(v) 
pprint.pprint(inter) 

#related_articles ? 

#intermediate = {}
#for k,v in a.items():
intermediate = get_ordered_categories_of_most_related_people(inter)

pprint.pprint(intermediate) 


#tester is now a certain value of a
#a = get_ordered_categories_of_most_related_people(tester)
#pprint.pprint(OrderedDict(a)) 







#copy_new_ordered_dict_person = sorting_dict(copy_new_ordered_dict_person_test),
#pprint.pprint(person_questions_dict,indent=1, compact=True)
#pprint.pprint(copy_new_ordered_dict_person,indent=1, compact=True)
#pprint.pprint(get_related_with_categories(),indent=1, compact=True)


def get_related_with_categories():
  ret = {}

  person_questions_dict = dict(zip(person_df['Answer'], person_df['Question']))
  cat_ranking_person = get_categories(person_questions_dict)
  #pprint.pprint(cat_ranking_person, indent=1)
  cat_with_pv_person = get_pageviews_for_categories(cat_ranking_person)
  #pprint.pprint(cat_with_pv_person, indent=1)
  #for person: sorting the dict after pages per category
  categories_with_subs_and_pageviews_person = get_dict_for_every_location(cat_ranking_person, cat_with_pv_person)
  #pprint.pprint(categories_with_subs_and_pageviews_person,indent=1)
  new_ordered_dict_person = sorting_dict(categories_with_subs_and_pageviews_person)

  for key,value in new_ordered_dict_person.items():
    ret[key] = value


  return ret


  #related_articles = get_related_links(wiki_link)

#most_known_related_peoples_list = 


#pprint.pprint(cat_ranking_person)
#copy_new_ordered_dict_person


{'Max Verstappen': [{'url': 'https://en.wikipedia.org/wiki/Adrian_Newey', 'title': 'Adrian Newey'}, {'url': 'https://en.wikipedia.org/wiki/Alain_Prost', 'title': 'Alain Prost'}, {'url': 'https://en.wikipedia.org/wiki/Alberto_Ascari', 'title': 'Alberto Ascari'}, {'url': 'https://en.wikipedia.org/wiki/Alex_Albon', 'title': 'Alex Albon'}, {'url': 'https://en.wikipedia.org/wiki/Alex_Hitzinger', 'title': 'Alex Hitzinger'}, {'url': 'https://en.wikipedia.org/wiki/Alex_Zanardi', 'title': 'Alex Zanardi'}, {'url': 'https://en.wikipedia.org/wiki/Alexandre_Prémat', 'title': 'Alexandre Prémat'}, {'url': 'https://en.wikipedia.org/wiki/Andrew_Shovlin', 'title': 'Andrew Shovlin'}, {'url': 'https://en.wikipedia.org/wiki/Anthony_Kumpen', 'title': 'Anthony Kumpen'}, {'url': 'https://en.wikipedia.org/wiki/Antonio_Fuoco', 'title': 'Antonio Fuoco'}, {'url': 'https://en.wikipedia.org/wiki/Antonio_Giovinazzi', 'title': 'Antonio Giovinazzi'}, {'url': 'https://en.wikipedia.org/wiki/Ayrton_Senna', 'title': 'Ayrt

AttributeError: ignored

In [230]:
pprint.pprint(ordered_dict_related_person)
no = {}
for key,value in ordered_dict_related_person.items():
  #no[key] = dict(sorted(value.items(), key=lambda x: x[1][1], reverse=True))
  no[key] = OrderedDict(value)
pprint.pprint(no)

{'Joe Biden': OrderedDict([('https://en.wikipedia.org/wiki/Category:American_people_of_Irish_descent',
                            (100, 0, 32705)),
                           ('https://en.wikipedia.org/wiki/Category:American_people_of_English_descent',
                            (100, 0, 22035)),
                           ('https://en.wikipedia.org/wiki/Category:Presidents_of_the_United_States',
                            (100, 0, 18140)),
                           ('https://en.wikipedia.org/wiki/Category:American_Roman_Catholics',
                            (100, 0, 14103)),
                           ('https://en.wikipedia.org/wiki/Category:Presidential_Medal_of_Freedom_recipients',
                            (100, 0, 10987)),
                           ('https://en.wikipedia.org/wiki/Category:20th-century_American_politicians',
                            (100, 0, 8176)),
                           ('https://en.wikipedia.org/wiki/Category:American_people_of_French_descent',
 

In [203]:
"""
Function to :

1. retrieve all of the related links of a wiki-persons-page (first 500);
2. pre prune the list, such that only entries with 2 words in the title are left; (for performance reasons)
3. analyse the links that are left via wikipedia APi and the "instance of" (P31) property set to "human" (Q5)
4. now we have a list of related people to compare to our answer-person-entity

5. retrieve the categories of those related people as done above with the answer_entities
6. compare the categories and list the ones that appear in most often
7. look for the "unexpected"
"""

wiki_link = 'https://en.wikipedia.org/wiki/'


def unexpected_categories(person_answer_entity_list):
  wiki_link_list={}
  for val in person_answer_entity_list:
    names_underscores = val.replace(' ', '_')
    answer_entity_wiki_link = wiki_link + names_underscores
    wiki_link_list[val] =  answer_entity_wiki_link
  
  categories = get_categories(wiki_link_list)
  categories_with_pv = get_pageviews_for_categories(categories)
  categories_with_subs_pvs = get_dict_for_every_location(categories, categories_with_pv)

  test1 = check_inner_tuple_length(categories_with_subs_pvs )
  test2 = prune_and_ordered_dict(test1, 20)
  ordered_categories_dict_AnswerPeopleEntities = sorting_dict(test2)
  #up to this point, the function retrieves,pre-prunes and orderes the most popular categories from the person-answer-entities
  #now wee need to do the same for each of the selected_related_people_list, to then compare the categories and rank them from most to least occuring

  return(ordered_categories_dict_AnswerPeopleEntities)

def related_people_with_categories(categs_list):
  selected_people = {}
  selected_people_with_categories = {}
  for key, value in categs_list.items():
    names_underscores = key.replace(' ', '_')
    related_articles = get_related_links(wiki_link + names_underscores)
    filtered_links = filter_two_word_titles(related_articles)
    related_people_list = check_if_person(filtered_links)
    select_random_people = select_random_entries(related_people_list)

    selected_people[key] = select_random_people


  #for key, value in selected_people.items():
  #  for title, url in value.items():

      #selected_people_with_categories[key] = get_categories

  return selected_people
    # now calculate the most popukar categories of the selected_people and compare them

  # print(wiki_link_list)  

person_answer_entity_list = list(person_df['Answer'])
#print(person_answer_entity_list)
#unexpected_categories(person_answer_entity_list)
#print(unexpected_categories(person_answer_entity_list))
a = unexpected_categories(person_answer_entity_list)
pprint.pprint(unexpected_categories(person_answer_entity_list))
b = related_people_with_categories(a)
pprint.pprint(b)
#print(person_questions_dict)
#10 mins

{'Joe Biden': OrderedDict([('https://en.wikipedia.org/wiki/Category:American_people_of_Irish_descent',
                            (100, 0, 32705)),
                           ('https://en.wikipedia.org/wiki/Category:American_people_of_English_descent',
                            (100, 0, 22035)),
                           ('https://en.wikipedia.org/wiki/Category:Presidents_of_the_United_States',
                            (100, 0, 18140)),
                           ('https://en.wikipedia.org/wiki/Category:American_Roman_Catholics',
                            (100, 0, 14103)),
                           ('https://en.wikipedia.org/wiki/Category:Presidential_Medal_of_Freedom_recipients',
                            (100, 0, 10987)),
                           ('https://en.wikipedia.org/wiki/Category:20th-century_American_politicians',
                            (100, 0, 8176)),
                           ('https://en.wikipedia.org/wiki/Category:American_people_of_French_descent',
 

In [207]:
def check_inner_tuple_length(pv_and_pages_dict):
  # Iterate over the dictionary
  for key, value in pv_and_pages_dict.items():
    # Iterate over the ordered dict values
    for inner_key, inner_value in value.items():
      # Check if the inner tuple has two elements
      if len(inner_value) == 2:
        # Update the value by adding a zero at the end
        value[inner_key] = inner_value + (0,)
  return pv_and_pages_dict

#test1 = check_inner_tuple_length(copy_categories_with_subs_and_pageviews_person )
#test2 = prune_and_ordered_dict(test1, 20)
#test3 = sorting_dict(test2)
#pprint.pprint(test3)

def with_categories(related_people_list):
  dict_with_categs = {}
  for key, value in related_people_list.items():
    related_people_with_categs={}
    for title, url in value:
      related_people_with_categs[title] = get_categories(url)
    dict_with_categs[key] = related_people_with_categs

#common_categories = find_most_common_links(a)
#print('These are the categories that occur most between the person-entities:')
#pprint.pprint(common_categories, indent=1, compact=True)
print(person_questions_dict)
print(b)
new_related_people_list = {}
for key, value in b.items():
  inner_list={}
  for item in value:
    inner_list[item['title']] = item['url']
  new_related_people_list[key] = inner_list

print(new_related_people_list)

# new_related_people_list_with_categs = {}
# for key, value in new_related_people_list.items():
#   #new_related_people_list_with_categs[key] = get_categories(value)
#   new_related_people_list_with_categs[key] = get_categories_with_pageviews_person(value)

# pprint.pprint(new_related_people_list_with_categs)
#l = with_categories(b)
#print(l)

{'Max Verstappen': 'Who was the F1 World Champion in 2022?', 'Michael Jackson': 'Who is the artist behind the song Thriller?', 'Joe Biden': 'Who is the current president of the US?'}
{'Max Verstappen': [{'url': 'https://en.wikipedia.org/wiki/Esteban_Ocon', 'title': 'Esteban Ocon'}, {'url': 'https://en.wikipedia.org/wiki/Lando_Norris', 'title': 'Lando Norris'}, {'url': 'https://en.wikipedia.org/wiki/Jean_Alesi', 'title': 'Jean Alesi'}, {'url': 'https://en.wikipedia.org/wiki/Hannah_Schmitz', 'title': 'Hannah Schmitz'}, {'url': 'https://en.wikipedia.org/wiki/Liam_Lawson', 'title': 'Liam Lawson'}], 'Michael Jackson': [{'url': 'https://en.wikipedia.org/wiki/Amy_Winehouse', 'title': 'Amy Winehouse'}, {'url': 'https://en.wikipedia.org/wiki/Barry_Mann', 'title': 'Barry Mann'}, {'url': 'https://en.wikipedia.org/wiki/Anthony_Newley', 'title': 'Anthony Newley'}, {'url': 'https://en.wikipedia.org/wiki/Bob_Newhart', 'title': 'Bob Newhart'}, {'url': 'https://en.wikipedia.org/wiki/David_Byrne', 'titl

In [202]:
pprint.pprint(a)
pprint.pprint(new_related_people_list_with_categs)

# stupid = {}
# for key,value in a.items():
#   stupid[key] = value

# for key, value in new_related_people_list_with_categs.items():
#   stupid[key] = value

# print(stupid)

# category_occurencess = find_most_common_links(stupid)

# pprint.pprint(category_occurencess)

ocus = {}
for key,value in new_related_people_list_with_categs.items():
  category_occurencess = find_most_common_links(value)
  ocus[key]= category_occurencess

pprint.pprint(ocus)


{'Joe Biden': {'Akiko Iwasaki': OrderedDict([('https://en.wikipedia.org/wiki/Category:1970_births',
                                              (100, 0)),
                                             ('https://en.wikipedia.org/wiki/Category:American_academics_of_Japanese_descent',
                                              (100, 0)),
                                             ('https://en.wikipedia.org/wiki/Category:American_immunologists',
                                              (100, 0)),
                                             ('https://en.wikipedia.org/wiki/Category:Fellows_of_the_American_Academy_of_Microbiology',
                                              (100, 0)),
                                             ('https://en.wikipedia.org/wiki/Category:Howard_Hughes_Medical_Investigators',
                                              (100, 0)),
                                             ('https://en.wikipedia.org/wiki/Category:Living_people',
               

# **Old predictions methods**

## Functions for reuse

In [ ]:
#get Wikidata ID
def getQitemOfName(entity):
  
  searched = "'"+entity+"'"
  sparql.setQuery('''
  SELECT ?item ?itemLabel WHERE {
    ?item rdfs:label '''+searched+'''@en.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }

  }
  ''')
  
  sparql.setReturnFormat(JSON)
  entities = sparql.query().convert()
  entities_df = pd.json_normalize(entities['results']['bindings'])
  words = entities_df[["item.value"]].iloc[0].str.split("/")[0]
  return words[-1]

In [ ]:
#get name of entity from Wikidata ID

def getLabelOfQitems(entities):

  finalLabels = []
  for item in entities:
    #searched = "'"+item+"'"
    sparql.setQuery('''
    SELECT * WHERE {
        wd:'''+item+ ''' rdfs:label ?label .
        FILTER (langMatches( lang(?label), "EN" ) )
    } LIMIT 1
    ''')
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])
    label = entities_df["label.value"].iloc[0]
    finalLabels.append(label)

  return finalLabels  

In [ ]:
#function to get all backlinks to wikipage and saves backlinks >= 1000 in array
#source: https://www.mediawiki.org/wiki/API:Backlinks#Response

import requests

def getBacklinks(entities):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  listWithBacklink500 = []
  dictWithAllEntries = {}

  for val in entities:
    #print(row["itemLabel.value"])
    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": val,
        "bllimit": "max"
    }

    try:
      R = S.get(url=URL, params=PARAMS)
      DATA = R.json()
   

      BACKLINKS = DATA["query"]["backlinks"]

      count = 0
      for links in BACKLINKS:
        count +=1

      
      while "continue" in DATA:
        if (count >= 1000):
          break;
        blcontinue = DATA["continue"]["blcontinue"]
        PARAMS["blcontinue"] = blcontinue

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        BACKLINKS = DATA["query"]["backlinks"]

        for links in BACKLINKS:
          count += 1
      
      dictWithAllEntries[val] = count
      #print(val + ": " + str(count))

    except ValueError:
      print("Value Error")

    if (count >= 1000):
      listWithBacklink500.append(val) 

  if (len(listWithBacklink500) > 0):
    #return getPageViews(listWithBacklink500)
    return listWithBacklink500

  else:
    listMostBacklinks = []
    sortedDict = dict(sorted(dictWithAllEntries.items(), key=lambda item:item[1]))

    for k, v in sortedDict.items():
      listMostBacklinks.append(k)
    
    numberToPass = len(listMostBacklinks)//3
        
    return listMostBacklinks[-numberToPass:]

In [ ]:
#function to get all backlinks to wikipage and saves backlink == 500 in array
#source: https://www.mediawiki.org/wiki/API:Backlinks#Response

import requests

def getBacklinksDict(entities):
  S = requests.Session()

  URL = "https://en.wikipedia.org/w/api.php"

  dictWithBacklink500 = {}
  dictWithAllEntries = {}

  for key, val in entities.items():
    print(key, val)

    PARAMS = {
        "action": "query",
        "format": "json",
        "list": "backlinks",
        "bltitle": key,
        "bllimit": "max"
    }

    try:
      R = S.get(url=URL, params=PARAMS)
      DATA = R.json()
   

      BACKLINKS = DATA["query"]["backlinks"]

      count = 0
      for links in BACKLINKS:
        count +=1

      
      while "continue" in DATA:
        if (count >= 1000):
          break;
        blcontinue = DATA["continue"]["blcontinue"]
        PARAMS["blcontinue"] = blcontinue

        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        BACKLINKS = DATA["query"]["backlinks"]

        for links in BACKLINKS:
          count += 1
      
      dictWithAllEntries[key] = count
      #print(val + ": " + str(count))

    except ValueError:
      print("Value Error")
    except KeyError:
      print("Key Error")

    if (count >= 1000):
      dictWithBacklink500.update({key: val}) 

  if (len(dictWithBacklink500) > 0):
    #return getPageViews(listWithBacklink500)
    return dictWithBacklink500

    #print(listMostBacklinks[-numberToPass:])

    
    return listMostBacklinks[-numberToPass:]

In [ ]:
#get avg pageViews of entities

def getPageViewsList(listEntities):
  avgDict = {}
  
  for val in listEntities:
    try:
      resp = pageviewapi.per_article('en.wikipedia', val, '20191106', '20201120', access='all-access', agent='all-agents', granularity='monthly')
    #print(resp)
      avgViews = 0
      count = 0
      for i in resp.get("items"):
        #print(i.get("views"))
        avgViews += i.get("views")
        count += 1
      avgDict.update({val: avgViews//count})
    except:
      print("Page not found to check pageViews for page: " + val)
      
  #return (max(avgDict, key=avgDict.get))
  return avgDict


  


In [ ]:
#get avg pageViews of entity
import pageviewapi


def getPageViews(entity):
  avgDict = {}
  
  try:
    resp = pageviewapi.per_article('en.wikipedia', entity, '20191106', '20201120', access='all-access', agent='all-agents', granularity='monthly')

    avgViews = 0
    count = 0
    for i in resp.get("items"):
      #print(i.get("views"))
      avgViews += i.get("views")
      count += 1

    pageViews = avgViews//count
    #avgDict.update({val: avgViews//count})

    #return (max(avgDict, key=avgDict.get))
    return pageViews
  #print(resp)
  except:
    print("Page not found to check pageViews for page: " + entity)
    return -1
    


In [ ]:
#check if entity is a person
#return list of persons

def checkIfHuman(entity):

  #listHumans = []

  #for test in entity:
    #print(item)
  searched = "'"+entity+"'"

  #print(searched)

  try:

    sparql.setQuery('''
        SELECT distinct ?item ?itemLabel 
        WHERE{
          ?item ?label '''+searched+'''.  
          ?item wdt:P31 wd:Q5.
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.	
          SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }    
        }
    ''')
    sparql.setReturnFormat(JSON)
    entities = sparql.query().convert()
    entities_df = pd.json_normalize(entities['results']['bindings'])

    #print(entities)
    #print(entities_df["itemLabel.value"].item())

    #for index, row in entities_df.iterrows():
    return entities_df["itemLabel.value"].item()
    
    
  except:
    return None


In [ ]:
#if parantheses in entity -> remove

def removeParanthesesDict(inputDict):

  newDict = {}
  for key, val in inputDict.items():
    if ('(' in key):
      newElem = re.sub("[\(\[].*?[\)\]]", "", key).rstrip()
      newDict.update({newElem: val})
    else:
      newDict.update({key: val})

  return newDict

##Hints if answer is a Location

In [ ]:
#Query with all properties to get data

def allHintsLocation(entity):
  sparql.setQuery('''
  SELECT ?itemLabel ?property
  WHERE
  {  
    VALUES ?property {wdt:P35 wdt:P30 wdt:P36 wdt:P610 wdt:P1082 wdt:P421 wdt:P38 wdt:P17 wdt:P1376 wdt:P131 wdt:P6 wdt:P1830 wdt:P47 wdt:P206 wdt:P37 wdt:P463 wdt:793}
    wd:'''+entity+''' ?property ?item.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  hintsLocation = sparql.query().convert()

  hintsLocation_df = pd.json_normalize(hintsLocation['results']['bindings'])

  return hintsLocation_df

In [ ]:
#create template based hints

def getFormatedHintsLocation(queryResults_df):
  timeZone = []
  isLocatedIn = []
  ownerOf = []
  shareBorder = []
  bodyOfWater = []
  languages = []
  memberOf = []
  significantEvents = []

  listHintsLocation = []

  searchString = "http://www.wikidata.org/prop/direct/"
  
  for index, row in queryResults_df.iterrows():
    if(not location in row["itemLabel.value"]):
      if(row["property.value"] == searchString + "P35"):
        listHintsLocation.append("Head of state of searched country is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P30"):
        listHintsLocation.append("The searched location is on continent " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P36"):
        listHintsLocation.append("The capital of searched country is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + ""):
        listHintsLocation.append("The highest point in searched location is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P1082"):
        listHintsLocation.append("The searched location has a population of " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P38"):
        listHintsLocation.append("Currency in searched location is " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P17"):
        if(not location in row["itemLabel.value"]):
          listHintsLocation.append("The searched location is in country "+ row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P1376"):
        listHintsLocation.append("The searched city is capital of " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P131"):
        listHintsLocation.append("The searched location is located in " + row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P6"):
        listHintsLocation.append("Head of government of searched location is " + row["itemLabel.value"])
      
      #if get multiple values save all in list and pick one afterwards
      elif(row["property.value"] == searchString + "P1830"):
        if(not location in row["itemLabel.value"]):
          ownerOf.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P47"):
        shareBorder.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P206"):
        bodyOfWater.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P37"):
        if(not location in row["itemLabel.value"]):
          languages.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P421"):
        timeZone.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P463"):
        memberOf.append(row["itemLabel.value"])
      elif(row["property.value"] == searchString + "P793"):
        if(not location in row["itemLabel.value"]):
          significantEvents.append(row["itemLabel.value"])

  #getting more than one result with one query, pick one for output
  if(len(ownerOf) > 0):
    listHintsLocation.append("The searched location is owner of " + ownerOf[0])
  if(len(shareBorder) > 0):
    listHintsLocation.append("The searched location shares border with " + shareBorder[0])
  if(len(bodyOfWater) > 0):
    listHintsLocation.append("The next body of water of searched location is " + bodyOfWater[0])
  if(len(memberOf) > 0):
    listBacklinks = getBacklinks(memberOf)
    dictPageViews = getPageViewsList(listBacklinks)
    listHintsLocation.append("Searched location is member of " + (max(dictPageViews, key=dictPageViews.get)))
  if(len(significantEvents) > 0):
    listBacklinks = getBacklinks(significantEvents)
    dictPageViews = getPageViewsList(listBacklinks)
    listHintsLocation.append("A significant event happened in this location was " + (max(dictPageViews, key=dictPageViews.get)))

  #if getting more than one language concatenate all
  commaCounter = len(languages)-1
  stringLanguages = ""
  for language in languages:
    if(commaCounter > 0):
      stringLanguages += language + ", "
      commaCounter -= 1
    else:
      stringLanguages += language
  if(stringLanguages != ""):
    listHintsLocation.append("Spoken language(s) in searched location is/are " + stringLanguages)
  if(len(timeZone) > 0):
    listHintsLocation.append("The searched location is in time Zone " + timeZone[0])

  
  return listHintsLocation


In [ ]:
def writeLocationHintsInFile(listHintsLocation):
  randomHintsList = random.sample(listHintsLocation, len(listHintsLocation))
  
  for item in randomHintsList:
    with open("hintsLocation.txt", 'a') as writefile:
      writefile.write(item + "\n")
      writefile.close()

##Hints if answer is a Person

In [ ]:
#Query with all properties to get data

def allHintsPerson(entity):
  sparql.setQuery('''
  SELECT ?itemLabel ?property
  WHERE
  {  
    VALUES ?property {wdt:P21 wdt:P27 wdt:P569 wdt:P19 wdt:P1971 wdt:P106 wdt:P1340 wdt:P1884 wdt:P2048 wdt:P39 wdt:P69 wdt:P512 wdt:P102 wdt:P3602 wdt:P800 wdt:P166 wdt:P3373 wdt:P1412 wdt:P413 wdt:P118 wdt:P54}
    wd:'''+entity+''' ?property ?item.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  }
  ''')
  sparql.setReturnFormat(JSON)
  hintsPerson = sparql.query().convert()

  hintsPerson_df = pd.json_normalize(hintsPerson['results']['bindings'])
  
  return hintsPerson_df

In [ ]:
#create temolate based hints

def getFormatedHintsPerson(queryResults_df):

  listOccupation = ["Searched person was occuupied as "]
  listPositionHeld = ["Searched person held the position(s) "]
  listEducatedAt = ["Searched person was educated at "]
  listAcademicDegrees = ["Searched person has academic degrees "]
  listPoliticParty = ["Searched was/is member of politic party "]
  listCandidacyElection = ["Searched person was candidacy at election(s) "]
  listNotableWork = ["Searched person did notable work "]
  listAwardsReceived = ["Searched person received awards "]
  listLanguages = ["Searched person speaks language(s) "]
  listPlayedPositions = ["Searched person played position "]
  listLeague = ["Searched person played in leagues "]
  listTeams = ["Searched person played in teams "]
  listCitizenship = ["Searched person has following citizenships "]
  listSiblings = []
  listHair = []

  listHintsPerson = []

  for index, row in queryResults_df.iterrows():
    if(row["property.value"] == "http://www.wikidata.org/prop/direct/P21"):
      listHintsPerson.append("Gender of searched person is " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P569"):
      listHintsPerson.append("Searched person was born in " + row["itemLabel.value"][:4])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P19"):
      listHintsPerson.append("Searched person was born in " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1971"):
      listHintsPerson.append("Number of children of seared person is " + row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1340"):
      listHintsPerson.append("Searched person has " + row["itemLabel.value"] + " eyes")    
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P2048"):
      listHintsPerson.append("Searched persons height is " + row["itemLabel.value"])    

    
    #if get multiple values save all in list and pick one afterwards
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P27"):
      listCitizenship.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P106"):
      listOccupation.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P39"):
      listPositionHeld.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P69"):
      listEducatedAt.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P512"):
      listAcademicDegrees.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P102"):
      listPoliticParty.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P3602"):
      listCandidacyElection.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P800"):
      listNotableWork.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P166"):
      listAwardsReceived.append(row["itemLabel.value"])

    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1412"):
      listLanguages.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P413"):
      listPlayedPositions.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P118"):
      listLeague.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P54"):
      listTeams.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P3373"):
      listSiblings.append(row["itemLabel.value"])
    elif(row["property.value"] == "http://www.wikidata.org/prop/direct/P1884"):
      listHair.append(row["itemLabel.value"])

  #if too many results could be return check
  #if more thane 3 -> cut
  listResultsCut = [listPositionHeld, listAcademicDegrees, listPoliticParty, 
                 listCandidacyElection, listNotableWork, listLanguages, 
                 listPlayedPositions, listLeague, listTeams, listCitizenship, listAwardsReceived, listEducatedAt, listOccupation]
  for singleList in listResultsCut:
    cutResults(singleList, listHintsPerson)

  #just get one hair color
  if(len(listHair) > 0):
    listHintsPerson.append("The searched person has " + listHair[0])

  #get number of siblings
  if(len(listSiblings) > 0):
    listHintsPerson.append("The searched person has " + str(len(listSiblings)) + " siblings")  
  
  return listHintsPerson

In [ ]:
#if more results than 3 -> print 3 results + "and x others"

def cutResults(listResults, listHintsPerson):
  
  numResults = len(listResults)-1
  resultsLeft = numResults-3

  if (len(listResults) > 1):
    if(resultsLeft > 0):
      listHintsPerson.append(getConcatenationMultipleResults(listResults[0:4])+ " and " + str(resultsLeft) + " others")
    else:
      listHintsPerson.append(getConcatenationMultipleResults(listResults))

In [ ]:
#concatenate multiple results with comma

def getConcatenationMultipleResults(resultList):
  concatenation = ""
  commaCounter = len(resultList)-1
  for item in resultList:
    if(commaCounter == len(resultList)-1):
      concatenation += item
      commaCounter-=1
    elif(commaCounter > 0):
      concatenation += item + ", "
      commaCounter-=1
    elif(commaCounter == 0):
      concatenation += item
  
  return concatenation


In [ ]:
def writePersonHintsInFile(listHintsPerson):
  with open("hintsPerson.txt", 'a') as writefile:
    #writefile.write("\n\n---Hints for Person " + person + "---\n\n")
    randomHintsList = random.sample(listHintsPerson, len(listHintsPerson))
    for item in randomHintsList:
      writefile.write(item + "\n")
    writefile.close()

## Hints if answer is a Year

**Get all outlinks of wikipedia year page and pass to getBacklinks. Than look for return values on page.content and print line**

In [ ]:
import spacy
def hintsYear(year):

  page = wikipedia.page(year, auto_suggest=False)
  pageContent = page.content

  #get all links from year page
  dictPageLinks = getOutlinks2(year)
  #print(dictPageLinks)

  #get rid of unnesercery links
  filteredPageLinks = filterDict(dictPageLinks)

  
  #get backlinks of all links
  dictBacklink = getBacklinksDict(filteredPageLinks)
  #first200 = {k: filteredPageLinks[k] for k in list(filteredPageLinks)[:200]}
  #dictBacklink = getBacklinksDict(first200)

  #remove parantheses of links
  dictAdjusted = removeParanthesesDict(dictBacklink)

  #get all possible hint (all lines with a entity #backlinks > 1000)
  dictPossibleHints = findLineInTextEvent(dictAdjusted, year)

  with open('./resultsEvents.txt', 'a') as writefile:
    writefile.write("Question: " + questionYear + "("+ str(year) + ")---\n")

  #check if entity because of which line was found is subject
  #if it is not the subject -> drop
  dictFinalHints = getFinalHints(dictPossibleHints)

  #get rid of number of foundevents at end of line
  #need to put number at the and because it is dict and would be overwritten
  dictFinalHintsAdjusted = {}
  for key, val in dictFinalHints.items():
    dictFinalHintsAdjusted.update({key.split('///', 1)[0]: val})  

  #returns dict sorted after PageViews
  dictSortedPV = sortHintsNumberPVSubject(dictFinalHintsAdjusted)

  #get highest PageView
  maxPV = list(dictSortedPV.values())[0]

  #sort hints PageViews and Simscore combined
  sortHintsUtilityScore(questionYear, dictSortedPV, maxPV)

  listHumansNames = []

  for key, val in dictAdjusted.items():
    result = checkIfHuman(key)
    if(result != None):
      listHumansNames.append(result)

  #findLineInTextBirthDeath(listHumansNames, year)

**Function to find line of most important link in year content**

In [ ]:
import re

def findLineInTextEvent(entitiesEvent, year):

  page = wikipedia.page(year, auto_suggest=False)
  pageText = page.content
  splitText = []

  splitText = (re.split("== Events ==|== Births ==", pageText ))

  contentEvents = splitText[1]

  foundEvent = 0

  dictHintsEntity = {}

  for key, val in entitiesEvent.items():
    #if (foundEvent == 20):
      #break;

    for line in contentEvents.split("\n"):
      #if (foundEvent == 20):
        #break;

      if val in line:
        words = line.split()
        #counterEntitiesInLine = 0
        if(len(words) < 15):
          dictHintsEntity.update({line + "///" + str(foundEvent): val})
          #listHints.append(line)
          foundEvent += 1
          #break;
  return dictHintsEntity

In [ ]:
def findLineInTextBirthDeath(entitiesHuman, year):

  page = wikipedia.page(year, auto_suggest=False)
  pageText = page.content
  splitText = []

  splitText = (re.split("== Births ==|== Deaths ==", pageText ))

  print(splitText)

  contentBirths = splitText[1]
  contentDeaths = splitText[2]

  foundBirth = 0
  foundDeath = 0

  #open file and append results
  with open('./resultsBirthDeath.txt', 'a') as writefile:
    writefile.write("---Birth/Death in year " + str(year) + "---")

    for entity in entitiesHuman:
      #if (foundBirth == 3 and foundDeath == 3):
        #writefile.write("\n")
        #break;

      for line in contentBirths.split("\n"):
        #if (foundBirth == 3):
          #break;

        if entity in line:
          writefile.write("Found with entity " + entity + "\n")
          writefile.write("Birth: " + line + "\n")
          writefile.write("\n")
          foundBirth +=1
          break;

      for line in contentDeaths.split("\n"):
        #if (foundDeath == 3):
          #break;
        if entity in line:
          writefile.write("Found with entity " + entity + "\n")
          writefile.write("Death: " + line + "\n")
          writefile.write("\n")
          foundDeath += 1
          break;
    writefile.write("\n")
      

In [ ]:
def sortHintsNumberPVSubject(dictHints):

  with open('./resultsEvents.txt', 'a') as writefile: 

    dictFinalHintsPV = {}

    for key, val in dictHints.items():
      dictFinalHintsPV.update({key: getPageViews(val)})

    dictFinalHintsSortedPV = dict(sorted(dictFinalHintsPV.items(), key=lambda item: item[1], reverse=True))
    """writefile.write("\n--Final Hints sorted #PV---\n")
    for key, val in dictFinalHintsSortedPV.items():
      writefile.write(key + "(" + str(val) + " PageViews)\n")"""

  return dictFinalHintsSortedPV



In [ ]:


def sortHintsUtilityScore(question, dictFinalHintsSortedPV, maxPV):
  model = SentenceTransformer('bert-base-nli-mean-tokens')

  question_embedding = model.encode(question)

  alpha = 0.5

  #sentence_embeddings = model.encode(hints)

  dictHintsBERT = {}

  for key, val in dictFinalHintsSortedPV.items():
    sentence_embedding = model.encode(key)
    simScore = cosine_similarity([question_embedding], [sentence_embedding]).item()
    impScore = alpha * (val/maxPV) + (1-alpha) * simScore
    dictHintsBERT.update({key: impScore})

  with open('./resultsEvents.txt', 'a') as writefile:

    dictHintsBERT = dict(sorted(dictHintsBERT.items(), key=lambda item: item[1], reverse=True))
    writefile.write("\n--Final Hints sorted Utility Score---\n")
    for key, val in dictHintsBERT.items():
      writefile.write(key + "(" + str(val) + " Utility Score)\n")

  return dictHintsBERT
 

In [ ]:
def get_subject_phrase(doc):
    for token in doc:
        if ("subj" in token.dep_):
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [ ]:
def getFinalHints(dictPossibleHints):

  #nlp = spacy.load("en")
  nlp = spacy.load("en_core_web_sm")
  dictFinalHintsSubject = {}
  
  for key, val in dictPossibleHints.items():
    doc = nlp(key)
    subject = get_subject_phrase(doc)
    if (val in str(subject)):
      dictFinalHintsSubject.update({key: val})
      #listFinalHints.append(key)

  return dictFinalHintsSubject

In [ ]:

def getOutlinks2(year):

  URL = 'https://en.wikipedia.org/wiki/' + str(year)

  # Fetch all the HTML source from the url
  response = requests.get(URL)


  soup = bs4.BeautifulSoup(response.text, 'html.parser')

  listTextOfLinks = []
  listLinks = []

  dictLinkName = {}


  for link in soup.find_all("a"):
    listTextOfLinks.append(link.get_text())
    dictLinkName.update({str(link.get("href")).split('/')[-1].replace('_', ' '): link.get_text()})

  #newDict = filterDict(dictLinkName)

  #print(len(dictLinkName))
  #print(len(newDict))
  #print(newDict)

  return dictLinkName

In [ ]:
def filterDict(dictNameLink):

  patternYear = r"[1-3][0-9]{3}"
  patternCentury = r"[1-2][0-9]\w+.century"
  patternAd = r"AD.\d\d\d\d"
  patternMonth = r"[a-zA-Z]+\s\d+"
  patternMillennium = r".+millennium"
  patternHashtag = r"#"
  patternCalendar = r"calendar"
  patternCategory = r"category"
  patternList = r"list"
  patternTemplate = r"template"
  patternWikipedia = r"wikipedia"
  patternIndex = r"index"
  patternHtml = r"html"
  patternPercent = r"%"

  newDict = {}

  for key, val in dictNameLink.items():
    if(not re.match(patternYear, key) and
       not re.match(patternCentury, key) and 
       not re.match(patternAd, key) and 
       not re.match(patternMonth, key) and
       not re.search(patternHashtag, key) and
       not re.search(patternCalendar, key, re.IGNORECASE) and
       not re.search(patternCategory, key, re.IGNORECASE) and
       not re.search(patternList, key, re.IGNORECASE) and
       not re.search(patternTemplate, key, re.IGNORECASE) and
       not re.search(patternWikipedia, key, re.IGNORECASE) and
       not re.search(patternIndex, key, re.IGNORECASE) and
       not re.search(patternHtml, key, re.IGNORECASE) and
       not re.search(patternPercent, key) and
       not re.match(patternMillennium, key) and
       not val == "" and not val == " "):
      

      newDict.update({key: val})


  print(newDict)

  return newDict


## Call Functions


**Call hint function for all questions where answer is Location**

In [ ]:
for index, row in location_df.iterrows():
  location = row["Answer"]
  questionLocation = row["Question"]
  with open("hintsLocation.txt", 'a') as writefile:
    writefile.write("\n\n" + questionLocation + " (" + location + ")\n\n")
    writefile.close()


  #get Wikidata ID for SPARQL-Query
  locationQitem = getQitemOfName(location)

  #execute SPARQL-Query
  queryResults_df = allHintsLocation(locationQitem)

  #getFinalHints
  listHintsLocation = getFormatedHintsLocation(queryResults_df)    

  #write hints in file in random order
  writeLocationHintsInFile(listHintsLocation)

**Call hint function for all questions where answer is Person**



In [ ]:

for index, row in person_df.iterrows():
  person = row["Answer"]
  questionPerson = row["Question"]

  with open("hintsPerson.txt", 'a') as writefile:
    writefile.write("\n\n" + questionPerson + " ("+ person +")\n\n")
    writefile.close()
  
  #get Wikipedia ID for SPARQL-Query
  personQitem = getQitemOfName(person)

  #execute SPARQL-Query
  queryResults_df = allHintsPerson(personQitem)
  
  #get Final Hints
  listHintsPerson = getFormatedHintsPerson(queryResults_df)

  #write hints in file in random order
  writePersonHintsInFile(listHintsPerson)


**Call hint function for all question where answer is Year**

In [ ]:
for index, row in year_df.iterrows():
  year = row["Answer"]
  questionYear = row["Question"]
  hintsYear(row["Answer"])

# Pageview Test Functions


In [ ]:
import time
import requests

from selenium import webdriver
from bs4 import BeautifulSoup

def get_table_info(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    options = webdriver.FirefoxOptions()
    #options.headless = True
    options.add_argument('--headless')

    driver = webdriver.Firefox(options=options)
    driver.get(url)
    time.sleep(3) # Wait for the page to load completely
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])
    return (headers, data)

def get_table_info_requests(url):
    """
    Given a URL, this function opens the url and retrieves the information stored in a table.
    """
    headers = []
    data = []

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    headers = [header.text.strip() for header in rows[0].find_all('th')]
    data = []
    for row in rows[1:]:
        data.append([cell.text.strip() for cell in row.find_all('td')])

    return (headers, data)

def get_links_in_section(wikipedia_url, section_heading):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link on that section of the page.
    """
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('span', {'id': section_heading})
    if section is None:
        return None
    section_content = section.parent.find_next_sibling('ul')
    if section_content is None:
        return None
    links = []
    for item in section_content.find_all('li'):
        link = item.find('a')
        if link is not None and link.has_attr('href') and link['href'].startswith('/wiki/'):
            #title = link.get('title', '')
            title = link.get('title')
            description = item.text.strip()
            url = f"https://en.wikipedia.org{link['href']}"
            links.append({'title': title, 'description': description, 'url': url})
    #print(links)
    return links

def get_links_in_section_with_sublinks(wikipedia_url, section_title):
    """
    Given a Wikipedia URL, and a section of this article, returns an array of dictionaries containing the href, title, and description of each link  with its sublinks as well.
    """
    response = requests.get(wikipedia_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        section_heading = soup.find('span', {'id': section_title})
        if section_heading is None:
            return None
        section = section_heading.parent
        section_content = section.find_next_sibling('ul')
        if section_content is None:
            return None
        links = []
        for item in section_content.find_all('li'):
            link = item.find('a')
            if link is not None:
                link_title = link.get('title')
                link_url = 'https://en.wikipedia.org' + link.get('href')
                link_description = item.text.replace(link_title, '').strip()
                links.append({'title': link_title, 'url': link_url, 'description': link_description})
                for sublink in item.find_all('a', href=True, recursive=False):
                    sublink_title = sublink.get('title')
                    sublink_url = 'https://en.wikipedia.org' + sublink.get('href')
                    sublink_description = sublink.parent.text.replace(sublink_title, '').replace(link_title, '').strip()
                    links.append({'title': sublink_title, 'url': sublink_url, 'description': sublink_description})
        return links
    else:
        return None

def get_events_links(title):
    """
    Gets all the links in the "Events" section of a Wikipedia page.
    Returns a list of strings containing the URLs of each event.
    """
    url = f"https://en.wikipedia.org/w/api.php?action=parse&format=json&page={title}&prop=text"
    response = requests.get(url)
    data = response.json()

    soup = BeautifulSoup(data['parse']['text']['*'], 'html.parser')
    events_section = soup.find('span', {'class': 'mw-headline', 'id': 'Events'})
    if events_section:
        events_list = events_section.find_next('ul')
        if events_list:
            events = events_list.find_all('a')
            event_links = [f"https://en.wikipedia.org{event['href']}" for event in events]
            return event_links
    return None

def create_url_from_title(title):
    """
    Given a list of dictionaries containing href, title, and description for each entry, extracts the title from each link and constructs a new URL using the title.
    """
    urls = []
    if title != None: 
      # Replace spaces with underscores in the title
      title = title.replace(' ', '_')

      # Construct a new URL using the title
      url = 'https://pageviews.wmcloud.org/redirectviews/?project=en.wikipedia.org&platform=all-access&agent=user&range=all-time&sort=views&direction=1&view=list&page=' + title
    return url

def extract_visitorNumber_from_link(link):
    """
    extract the first entry of the table
    """
    headers, data = get_table_info(link)

    try:
      if data[1]: 
        return data[1][2]
      else:
        return 0
    except IndexError:
      print("Index out of range")

def get_visitor_numbers(pageviews_url):
    """
    returns visitor numbers of the page
    """
    response = requests.get(pageviews_url)
    if response.status_code == 200:
        data = response.json()['items']
        visitors = {}
        for item in data:
            visitors[item['timestamp']] = item['views']
        return visitors
    else:
        return None

In [ ]:
#TEST

#Seitenaufrufe
#url = 'https://pageviews.toolforge.org/?project=en.wikipedia.org&pages=Formula1|Michael_Schumacher'
#headers, data = get_table_info(url)
#print(headers)
#print(data)

#get all wiki links on a page -> discard the one we dont need -> analyse the ones that could be usefull -> sort them after the backlinks
#print(get_events_links(2004))

# Search the most popular births and deaths in that year and then order them 
url1 = 'https://en.wikipedia.org/wiki/Category:1994_births'
url2 = 'https://en.wikipedia.org/wiki/Category:1994_deaths'
url3 = 'https://en.wikipedia.org/wiki/1994#Events'

#links_events_section = get_links_in_section_with_sublinks(url3, 'Events')
#print(len(links_events_section))
#print(links_events_section)
#print(get_births(1994))

#visitor number test
link = 'https://pageviews.wmcloud.org/redirectviews/?project=en.wikipedia.org&platform=all-access&agent=user&range=all-time&sort=views&direction=1&view=list&page=North_American_Free_Trade_Agreement'
#print(extract_visitorNumber_from_link(link))

In [ ]:
#When did Michael Schumacher win his first F1 World Drivers Title? -> 1994
#First 1994 is searched on wikipedia, then all the links from the event sections of that page are filtered and written into a dict together with their pageviews.
#Now order these lists such that the entry with most pageviews is on top
#THIS SHOULD WORK

year=2004
url = 'https://en.wikipedia.org/wiki/2004'

links_events_section = get_links_in_section_with_sublinks(url, 'Events')
print(len(links_events_section))

for link in links_events_section:
    t_title =  link.get('title') 
    t_url = create_url_from_title(t_title)
    link["pageview_url"] = t_url
    link["pageviews"] = extract_visitorNumber_from_link(t_url)

#sort the list links_event_section after the pageviews such that the entity with the highest pageviews is first
sorted_list = sorted(links_events_section, key=lambda x: int(x['pageviews'].replace(',', '')), reverse=True)

In [ ]:
import pprint
pageview_test_generated_hints_for_years = sorted_list

pprint.pprint(pageview_test_generated_hints_for_years, indent=4)

# Clean GitHub directory




In [ ]:
%cd /content/
%rm -r optimizationOfHintGeneration
%cd /content/
%rm hintsLocation.txt
%rm hintsPerson.txt
%rm resultsEvents.txt

/content
/content
rm: cannot remove 'hintsLocation.txt': No such file or directory
rm: cannot remove 'hintsPerson.txt': No such file or directory
rm: cannot remove 'resultsEvents.txt': No such file or directory
